In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output

In [ ]:
import json
biorxiv_medrxiv={}
for dirname, _, filenames in os.walk('/kaggle/input/CORD-19-research-challenge/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv/'):
    for filename in filenames:
        op=open(os.path.join(dirname,filename))
        f=json.load(op)
        biorxiv_medrxiv[os.path.join(filename)]=f

In [ ]:
abstrac=[]
bod=[]
tit=[]
for k in biorxiv_medrxiv.keys():
    abstract=[]
    body=[]
    title=[]
    lens=len(biorxiv_medrxiv[k]['abstract'])
    bodl=len(biorxiv_medrxiv[k]['body_text'])
    
    
    title.append(biorxiv_medrxiv[k]['metadata']['title'])
        
    for i in range(0,lens):
        abstract.append(biorxiv_medrxiv[k]['abstract'][i]['text'])
    for j in range(0,bodl):
        body.append(biorxiv_medrxiv[k]['body_text'][j]['text'])
    abstrac.append(' '.join(abstract))
    bod.append(' '.join(body))
    tit.append(' '.join(title))
    
    
    

In [ ]:
custom={}
for dirname, _, filenames in os.walk('/kaggle/input/CORD-19-research-challenge/2020-03-13/pmc_custom_license/pmc_custom_license/'):
    for filename in filenames:
        op=open(os.path.join(dirname,filename))
        f=json.load(op)
        custom[os.path.join(filename)]=f

In [ ]:
abstrac2=[]
bod2=[]
tit2=[]
for k in custom.keys():
    abstract2=[]
    body2=[]
    title2=[]
    lens=len(custom[k]['abstract'])
    bodl=len(custom[k]['body_text'])
    
    
    title2.append(custom[k]['metadata']['title'])
        
    for i in range(0,lens):
        abstract2.append(custom[k]['abstract'][i]['text'])
    for j in range(0,bodl):
        body2.append(custom[k]['body_text'][j]['text'])
    abstrac2.append(' '.join(abstract2))
    bod2.append(' '.join(body2))
    tit2.append(' '.join(title2))
    
    
    

In [ ]:
tif=tit+tit2
abstrf=abstrac+abstrac2
bodf=bod+bod2

In [ ]:
daaw=pd.DataFrame({'title':tif,'abstracts':abstrf,'body':bodf})

In [ ]:
daa=daaw.copy()

## The 'daa' DataFrame contain the titles, abstracts and body text from all CORD-19-research-challenge data.

Now we go onto Data Cleaning .

In [ ]:
from string import punctuation
pun=list(punctuation)
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
sbst=SnowballStemmer('english')
stwo=stopwords.words('english')
from nltk.tokenize import word_tokenize
def clen(s):
    for j in pun:
        ss=s.lower().split(j)
        s=' '.join(ss)
    toks=word_tokenize(s)
    cle=[sbst.stem(w) for w in toks if w not in stwo]
    return ' '.join(cle)
daa['abstracts']=daa['abstracts'].apply(clen)
daa['body']=daa['body'].apply(clen)
daa['title']=daa['title'].apply(clen)


## Creating Document-Term Matrix : for word wise analaysis.

In [ ]:
# DTM(onegram)

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=20000).fit(daa['abstracts']+daa['body'])
sk=cv.transform(daa['abstracts']+daa['body'])
tDTM=pd.DataFrame(sk.toarray(),columns=cv.get_feature_names())

## Plot - 1 : Word Counts

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
sb.set(style='white')

In [ ]:
wc1gr=tDTM.sum().sort_values(ascending=False).reset_index()


In [ ]:
plt.figure(figsize=(21,7))
sb.barplot(x='index',y=0,data=wc1gr.iloc[:60,:])
plt.xticks(rotation=60)
plt.tick_params(axis='x',which='major',labelsize=20)
plt.tight_layout()

In [ ]:
# DTM(twogram)

from sklearn.feature_extraction.text import CountVectorizer
cv2=CountVectorizer(max_features=20000,ngram_range=(2,2)).fit(daa['abstracts']+daa['body'])
sk2=cv2.transform(daa['abstracts']+daa['body'])
tDTM=pd.DataFrame(sk2.toarray(),columns=cv2.get_feature_names())



In [ ]:
wc2gr=tDTM.sum().sort_values(ascending=False).reset_index()

In [ ]:
plt.figure(figsize=(21,7))
sb.barplot(x='index',y=0,data=wc2gr.iloc[:60,:])
plt.xticks(rotation=60)
plt.tick_params(axis='x',which='major',labelsize=20)
plt.tight_layout()

## The Documents are to be Clustered for Topic Wise Analysis.

### We are using the Term-Frequency Inverse Document Frequency Matrix for K Means Clustering

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfmo=TfidfTransformer().fit(sk)
tfs=tfmo.transform(sk)
tfDTM=pd.DataFrame(tfs.toarray(),columns=cv.get_feature_names())


In [ ]:
from sklearn.cluster import KMeans
kmeans=KMeans().fit(tfDTM)
clus=kmeans.predict(tfDTM)

In [ ]:
centword={}
i=1
for k in kmeans.cluster_centers_:
    ind=k.argsort()[::-1][:50]
    centword[i]=tfDTM.columns[ind]
    i=i+1


## Now we know the relevent Topic Names used frequently for reference.



## Searching for Relevent topics :

### Before Searching we need to find the Relevent Keywords from Centword Dictionary.


In [ ]:
print(centword)

In [ ]:
## we can get the stemmed format of a word using
sbst.stem('genetic')

In [ ]:
tfDTM['sum']=tfDTM['viral']+tfDTM['genet']+tfDTM['evolut']+tfDTM['covid']+tfDTM['manag']+tfDTM['origin']\
                + tfDTM['transmiss']+tfDTM['distribut']+tfDTM['symptom']

so=tfDTM.sort_values(by='sum',ascending=False).index
d=[]
c=[]
t=[]
for k in so[:5]:
    d.append(daaw['body'].iloc[k])
    c.append(daaw['abstracts'].iloc[k])
    t.append(daaw['title'].iloc[k])
print(len(' '.join(c)))


In [ ]:

for i in range(0,3):
    print(str(i+1)+'.',t[i],'\n')
    print('Abstract:',c[i],'\n')
    print('Body:',d[i][:10000])
    
    print('\n')

# What do we know about virus genetics, origin, and evolution?

1. Prevalence and clinical features of 2019 novel coronavirus disease (COVID-19) in the Fever Clinic of a teaching hospital in Beijing: a single-center, retrospective study 

Abstract: Background With the spread of COVID-19 from Wuhan, Hubei Province to other areas of the country, medical staff in Fever Clinics faced the challenge of identifying suspected cases among febrile patients with acute respiratory infections. We aimed to describe the prevalence and clinical features of COVID-19 as compared to pneumonias of other etiologies in a Fever Clinic in Beijing. 

Body: The first cluster of cases of pneumonia with unknown origin emerged in the city of Wuhan, Hubei Province, China, in early December, 2019 1,2 . A novel coronavirus had been isolated from these patients, and subsequently named as 2019 novel coronavirus (2019-nCoV) on January 12, 2020 3 , and later as SARS-CoV-2. The initially confirmed cases were localized in Wuhan and mostly associated with Huanan Seafood Wholesale market in the city 2 , but soon the disease spread rapidly to other areas of the country. On January 19, 2020, two cases in Beijing and one case in Guangdong Province were reported 4 , and subsequently more cases were reported elsewhere 5-8 . Since the outbreak in Wuhan, as a quick response, Fever Clinics were requested to reinforce case finding of pneumonia of unknown etiologies. Fever Clinics, mostly affiliated to the Division of Infectious Diseases in general hospitals, had been established after SARS outbreak in 2003, and since then served as the first line to monitor and manage acute febrile respiratory infections, mostly seasonal influenza in recent years. For 2019 novel coronavirus disease (COVID-19), this time, patients with body temperatures≥37.2 o C were asked to firstly visit Fever Clinics, where a triage strategy was implemented 9 , and probable or possible cases were identified by experts or expert panels, and samples were sent for quick viral detection. On Jan. 21, 2020, the first case of COVID-19 was identified in our Fever Clinic, and up to Feb. 15, 21 cases were diagnosed, with 19 cases confirmed by positive 2019-nCoV results, among 88 patients with pneumonia sent for viral detection. As far as we know, the prevalence of COVID-19 in patients with pneumonia visiting Fever Clinics has not been reported, and there is lack of data comparing the clinical features between COVID-19 and pneumonia of other etiologies. Therefore, we collected and reviewed the medical records of patients who visited the Fever Clinic in our hospital. We aimed to estimate the prevalence of COVID-19 in pneumonias during this period and to find the unique features of COVID-19 as compared to pneumonias caused by other agents. We believe that the results of our study will add to the knowledge of COVID-19 for practice in Fever Clinics facing newly emerging respiratory infections. We conducted a retrospective, single-centered study and recruited patients visiting the Fever Clinic at Peking University Third Hospital from January 21 to February 15, 2020. Based on epidemiological history, clinical and radiological manifestations, cases with possible or probably COVID-19 were sent for panel discussion and then for 2019-nCoV detection by RT-PCR. Pediatric patients were not included in our study. The study protocol was approved by the ethics committee of Peking University Third Hospital and the data were analyzed anonymously. The  From January 21 to February 15, 2020, based on epidemiological evidence, fever and/or respiratory symptoms, chest radiological findings and blood white blood cell (WBC) results, physicians at the Fever Clinic referred 156 cases to panel discussion by multi-discipline experts. After discussion, 110 were considered to be possible or probable cases of COVID-19, and received 2019-nCoV real-time PCR testing, which was positive in 19 cases (confirmed cases). In another 2 patients, though PCR testing was negative, a clinical diagnosis was made according to epidemiological evidence, consistent clinical and CT findings (clinical cases). These 21 cases were grouped as COVID-19 for this analysis. For the remaining cases with negative viral detection, the diagnosis of COVID-19 was excluded based on inconsistent epidemiological, clinical or radiological data. Among these, 22 were excluded from the analysis because of lack of CT scan or no signs of pneumonia on CT scan, and finally 67 patients with pneumonia of other etiologies were included for analysis as the non-COVID-19 group (see patient selection flowchart in Figure 1 ). In the study period, 2123 patients visited the Fever Clinic because of fever and/or respiratory symptoms, and 342 patients were diagnosed to have pneumonia based on CT scan or in a few cases on Chest X-ray. The prevalence of COVID-19 in all these pneumonia patients was 6.14% (21/342). The prevalence of COVID-19 in cases with CT-confirmed pneumonia sent for 2019-nCoV testing was 23.9% (21/88). The demographics and baseline characteristics of the 21 patients with COVID-19 were shown in Table 1 . The median age was 42.0 years (25th-75th percentile, 34.5-66.0 years). Men and women were equally affected. Nineteen patients (90.5%) had a clear epidemiological history All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . https://doi.org/10.1101/2020.02.25.20027763 doi: medRxiv preprint of COVID-19. Seven patients, from 5 family clusters, had close contact with their family members. In the two patients with unclear epidemiological link, one was a social worker contacting a high-flow of people and his wife was confirmed to have COVID-19 by positive 2019-nCoV test about one week later. The other one had no epidemiological history on presentation, but another patient, who had had dinner with him, was also diagnosed to have COVID-19 later. An incubation period was elicited from 12 patients (57.1%), ranging from 2 to 10 days with a median of 6.5 days. Most of the patients were mild-moderate in disease severity (mild-moderate cases, n=18, 85.7%). Three patients were classified as severe cases with SpO 2 ≤ 93% on presentation, while no very severe cases were identified in our series. On presentation, most patients (85.7%) had fever with a mean body temperature of 37.8 o C. Cough (42.9%), expectoration (33.3%), fatigue (57.1%), headache or dizziness (38.1%) were common symptoms. Other symptoms included shortness of breath, myalgia or arthralgia, sore throat, nasal symptoms and diarrhea ( Blood routine showed that decreased lymphocyte count was present in 38.1% of the COVID-19 patients. More than 80% of COVID-19 patients had normal WBC and neutrophil counts. Compared with COVID-19 patients, the total WBC and neutrophil counts were higher in non-COVID-19 patients. Additionally, more non-COVID-19 patients had higher than normal WBC and neutrophil counts. However, lymphocyte counts and the proportions of patients with decreased lymphocytes were not statistically different between the two groups (Table 3 ). In non-COVID-19 patients, infiltrates on the CT scan mainly involved 1-2 lung lobes, while in COVID-19 patients, lesions involving 4-5 lobes were more common. In COVID-19 patients, lung lesions were mostly peripheral or sub-pleural in distribution, and in severe cases were diffusely distributed (Figure 2 ). In non-COVID-19 patients, 34.3% showed airway-dominant lesions, while diffuse distribution was found in only 4.5%. The patterns of the lesions showed no significant difference between the two groups. However, centrilobular nodules were observed in non-COVID-19 patients but not in COVID-19 patients (Table 4 ). As of Feb 15, 2020, 57416 cases of confirmed COVID-19 were reported in 31 provinces, autonomous regions and municipalities in China, among which 49030 were in Hubei Province, while in Beijing municipality, 380 confirmed cases were reported. Since the outbreak in early December, 2019, case finding and reporting of COVID-19 were required for medical staff in Fever Clinics around the country. During this study period, in Beijing, as in other regions of the country, influenza and other respiratory infections were also prevalent; it was a challenge for physicians to identify suspected COVID-19 from patients with fever, particularly from pneumonia of varied etiologies. Our results here provide information on the prevalence of COVID-19 in the busy Fever Clinic of a teaching hospital and differential characteristics from non-COVID-19 pneumonias. From January 21, 2019, when the first case of COVID-19 was identified, up to Feb.15, 2020, 2123 patients visited our Fever Clinic because of fever and/or respiratory symptoms, and 342 patients were confirmed to have pneumonia by CT scan or in a few cases by Chest X-ray. The prevalence of COVID-19 in all these pneumonia patients was 6.14% (21/342). The prevalence of All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . https://doi.org/10.1101/2020.02.25.20027763 doi: medRxiv preprint COVID-19 in cases with pneumonia sent for 2019-nCoV testing was 23.9% (21/88). We found that epidemiological history was extremely important for identification of suspected cases of COVID-19 and for differential diagnosis. Over 90% of the COVID-19 patients had clearly identified epidemiological evidence, while in non-COVID-19 patients, epidemiological links could only be found in 32.8%. In our series, most cases were imported from the city of Wuhan or Hubei Province, including traveling to or living in Wuhan or other cities in Hubei, and a few had a history of close contact with individuals coming from the above areas. In the later phase of our study, contact with family members with confirmed COVID-19 was an epidemiological feature and five family clusters were found. A report from Chinese CDC showed that the incidence of COVID-19 outside Hubei Province peaked in Jan 24-27 and most cases (68.6%) had ever exposed to Wuhan or Hubei within 14 days 11 . Weight of exposure history related to Wuhan and Hubei in confirmed cases was decreasing after February 11 . Consistent to previous reports, the incubation period of our cases was in a range of 2-10 days, with a median of 6.5 days [12] [13] [14] . A unique feature of our study was the comparison between COVID


2. Transmission and epidemiological characteristics of Severe Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2) infected Pneumonia (COVID-19): preliminary evidence obtained in comparison with 2003-SARS 

Abstract: [Abstract] Objectives Latest epidemic data of Severe Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2) infected Pneumonia (COVID-19) was collected and a detailed statistical analysis was carried out to make comparison with 2003-SARS in order to provide scientific reference for the prevention and control of COVID-19. All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. 

Body: report of an unexplained pneumonia patient. Subsequently, NHCPRC appointed an expert group to arrive in Wuhan to guide the epidemic management. On January 7, the pathogen of this unexplained infected pneumonia was identified as a novel coronavirus (2019-nCoV) [1] , which was renamed as Severe Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2) on February 12 th , 2020. SARS-CoV-2 could causes Coronavirus disease 2019 (COVID-19), also known as SARS-CoV-2 infected Pneumonia. On February 7 th , 2020, a total of 31774 cases of COVID-19 patients were diagnosed and reported in 31 provinces (autonomous regions, municipalities) and Xinjiang Production and Construction Corps (XPCC) [2] . At the same time, China's Hong Kong Special Administrative Region, Taiwan, Macau and Japan, the United All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. [3] [4] . Whether the effective control of 2003-SARS can provide direction for the prevention and control of COVID-19, there is still no reported evidence. In the present study, we collected relevant latest epidemic data of COVID-19 and a detailed statistical analysis was carried out and the results were compared with 2003-SARS in order to explore and provide scientific reference for the prevention and control of COVID-19. A confirmed COVID-19 patients was defined as a case with respiratory specimens that tested positive for the SARS-CoV-2 by at least one of the following three methods: isolation of SARS-CoV-2 or at least two positive results by real-time reverse-transcription-polymerase-chain-reaction (RT-PCR) assay for SARS-CoV-2 or a genetic sequence that matches SARS-CoV-2 [5] . All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . https://doi.org/10.1101/2020.01.30.20019836 doi: medRxiv preprint The data of confirmed COVID-19 cases from January 10 th , 2020 to February 7 th , 2020 and 2003-SARS cases from March 26 th to May 26 th , 2003 in China's inland regions (data from websites of NHCPRC and the World Health Organization) was collected and then the epidemic trends of the both infectious diseases were analyzed to understand the transmission dynamics of them. The basic information of COVID-19 patients reported in inland areas of China (Beijing, Shaanxi Province, Sichuan Province, etc.) was collected, including age, gender, and contact history (consist of having been to Hubei or living in Hubei, contacting confirmed cases, contacting people who were from Hubei and unclear). The data was collected from the website of the health and health committees of the above provinces. A descriptive epidemiological analysis method was employed to carefully analyze the epidemiological characteristics of COVID-19. However, only several provincial administrations released the baseline information of COVID-19 confirmed patients, so the number of confirmed cases with detailed baseline information was far less than the total. Finally, 853 COVID-19 confirmed patients from 18 provinces, whose baseline information was available, were included into our analysis. COVID-19 originated in Wuhan, China. In the early stage of COVID-19 outbreak, there may be cases of underreporting or concealment in Wuhan, which may make it difficult to objectively reflect the true epidemic characteristics of COVID-19. Therefore, the COVID-19 epidemic data in Wuhan and other inland regions of China was analyzed separately and compared to obtain more objective COVID-19 epidemic knowledge. As of 24:00 on February 7 th , NHCPRC has received a total of 31774 confirmed COVID-19 cases in in 31 provinces (autonomous regions, municipalities) and Xinjiang Production and Construction Corps (XPCC), 6401 severe cases, 722 deaths, All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the  Both the numbers of confirmed cases in Wuhan and other inland areas showed increasing trends. From January 24 th , 2020, the number of confirmed cases in areas other than Wuhan in inland China began to exceed Wuhan ( Figure 4A ); the fatality rate in Wuhan (1.52% -6.64%) was significantly higher than that of inland areas of China except for Wuhan (0% -2.64%) ( Figure 4B ). As of 17:00 on February 7 th , 2020, the basic information of 853 confirmed COVID-19 cases was collected on the official websites of 18 provincial and municipal health committees. 853 confirmed COVID-19 cases were aged 1 months to 94 years and the average age was (45.05 ± 17.22) years; 853 confirmed COVID-19 cases included 450 males (52.75%) and 403 females (47.25%), the gender ratio (M: F) was 1.12: 1 ( Figure 5 , Table 1 ). In the 853 confirmed COVID-19 patients, 199 cases reported their contact history, in which 172 patients (86.43%) had a history of having been to Hubei or living in Hubei, 19 patients (9.55%) had a history of contacting confirmed cases, 8 patients (4.02%) had a history of contacting people who were from Hubei ( Figure 6 ). In 2002, a case of SARS appeared in Guangdong Province, China, and then spread to Southeast Asia and all of the world [6] . The SARS outbreak comprehensively All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . https://doi.org/10.1101/2020.01.30.20019836 doi: medRxiv preprint tested that the public health system of China was very vulnerable in front of the prevention and control of infectious diseases. Subsequently, the Chinese government implemented a series of reforms in the public health field, such as reorganization of the CDC, training public health professionals, and establishing the disease information system covering the whole country, the establishment of a complete reporting system for infectious diseases and an excellent mechanism for handling public health emergencies, etc. After a development for 17 years, a complete public health system has been established in China and great progress has been made in handling public health emergencies such as infectious disease epidemics. After the outbreak of COVID-19 in 2020, the Chinese government adopted Although many details such as the source of the virus and the ability to spread All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . https://doi.org/10.1101/2020.01.30.20019836 doi: medRxiv preprint among people are still unknown, an increasing number of cases that have confirmed the human-to-human transmission of the virus. Studies have shown that the interpersonal transmission ability of COVID-19 corresponds to SARS, but its doubling time (6 to 7 days) is shorter than that of SARS (about 9 days) [1, 7] , and the passage interval of COVID-19 is also relatively short. The second generation of COVID-19 cases unrelated to Wuhan has gradually appeared, which has also contributed to a sharp increase in short-term confirmed cases, resulting in a serious situation of COVID-19 prevention and control, and thus it is now very difficult to predict the epidemic trend. Age and comorbidities (such as diabetes or heart disease) are independent predictors of poor SARS-CoV and MERS-CoV outcomes. The spread of SARS-CoV and MERS-CoV occurred to a large extent through super-transmission events [6] [7] . Currently, there are data suggesting that there may be super-transmission events in COVID-19 transmission, and the entire range of interpersonal transmission capabilities of COVID-19 remains unknown [8] [9] . In addition to the fragility of medical institutions in the face of the new coronavirus epidemic, the risk of infection complications among hospital populations is also greatly increased [10] [11] . What we need to do now is to continue to strengthen prevention and control measures, actively strengthen patient admission, diagnosis, and treatment. Our results showed the following epidemiological characteristics related to COVID-19: 1) The overall epidemic of COVID-19 is similar to that of 2003-SARS, but the total number of confirmed cases is higher than that of 2003-SARS. The fatality rate is significantly lower than that of 2003-SARS in the same period, and the cure rate is significantly higher than that of 2003-SARS, which suggests that the prognosis for COVID-19 is good. 2) The age of COVID-19 patients is mainly concentrated in the 30-50 years old (60.61%), and the male are more susceptible than the female (M: F 1.12: 1). This may be related to the fact that the adults at this age group are more active in social activities than other age groups. All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . https://doi.org/10.1101/2020.01.30.20019836 doi: medRxiv preprint 8 3) The results showed that that the fatality rate of COVID-19 in Wuhan (1.52% -6.64%) was significantly higher than that in other regions (0%-2.64%). Most of the confirmed cases in Wuhan were the First-generation infections, and the other regions may exist more secondary cases. The information above provided evidence that the harm of the first-generation COVID-19 cases is significantly higher than that of secondary cases (including second-and third-generation cases). The  The authors declare that they have no competing interests. R. Z. and X. L. planned the study. author/funder, who has granted medRxiv a license to displ


3. Clinical Characteristics of 2019 Coronavirus Pneumonia (COVID-19): An Updated Systematic Review 

Abstract: Clinical characteristics of novel coronavirus pneumonia (COVID-19) have been described in numerous studies but yielded varying results. We aimed to conduct a systematic review on scientific literatures and to synthesize critical data on clinical traits of COVID-19 from its initial outbreak to pandemic. Systematic searches were conducted to identify retrospective observational study that contained clinical characteristics on COVID-19 through multiple databases. Two reviewers independently evaluated eligible publications. Data on clinical characteristics of COVID-19 were extracted and analyzed. RESULTS -two retrospective studies demonstrating the clinical characteristics of COVID-19 were included. The most common symptom was fever (2878 [83.0%]), and 63.4% of the patients presented fever as onset symptom. There were 2528 [88.2%] of 2866 cases had abnormal lung findings on chest CT scan. Laboratory findings showed that 1498 [62.8%] of 2387 cases had lymphopenia, and 1354 [64.8%] of 2091 cases had an increased level of C-reactive protein (CRP). A total of 185 [11.5%] patients were admitted to intensive care unit (ICU) while the overall case fatality rate (CFR) was 3.7%. Compared to patients admitted outside of Hubei, China, those from Hubei had a significant higher ICU admission rate (21.9% vs. 2.5%, p<0.001). Also, CFR attributed to COVID-19 was significantly higher in Hubei than that of non-Hubei admissions (10.4% vs. 0.6%, p<0.001). This large patient-based systematic review presents a more precise profiling of the COVID-19 from its outbreak to current pandemic. Dynamic evolvements of COVID-19 are needed to be characterized in future studies. 

Body: COVID-19 among different studies, partly due to anthropogenic differences in patients enrolled and differences in sample size across different studies. Notably, variation in reporting descriptive data may lead to the misunderstanding of COVID-9 characteristics. In this updated systematic review, hence, we sought to address the heterogeneities among published retrospective studies and to synthesize the available data. We expect this critical review will provide insights to understand the clinical characteristics of COVID-19 in a more systematic manner. Systematic searches were performed via the Medline database (PubMed) and Embase combining the terms (novel coronavirus OR 2019 novel coronavirus OR 2019-nCoV OR Coronavirus disease 2019 OR COVID-19 OR SARS-CoV-2). We also searched the database of Chinese Medical Journal full-text database (http://journal.yiigle.com/) for publications in Chinese using the above strategies. Searches were limited to publications from January 1, 2020 to March 1, 2020. Two of the authors (Z. F. and F. Y.) independently screened searching results to determine inclusion or exclusion of the articles. Disagreements were modulated by consulting another author as adjudicator (Z. L.). We included retrospective observational studies as long as they contain clinical characteristics of COVID-19 associated illness. If the patients came from the same hospital with overlapping cases, we only selected the publication containing greatest number of cases. The following All included literatures were evaluated using the Newcastle-Ottawa Scale (NOS) 11 . The quality score of the literature ranged from lowest 0 to highest 9. Total quality score of 0-3, 4-6, and 7-9 indicated poor, fair, and good studies, respectively. Information on baseline demographic data, medical and exposure history, symptoms and signs, underlying comorbidities, laboratory findings, chest computed tomographic (CT) scans and CFR were recorded. Based on the diagnostic gold standard for COVID-19 (positive RT-PCR assay for SARS-CoV-2), we synthesized data on demographic and clinical parameters in an unbiased manner. The proportion of each parameter was calculated by the following formula: (actual patient counts)/ (total patients confirmed using the gold standard) × 100%. The  A total of 1052 publications were retrieved using the search strategy. Seventy-two retrospective studies (including 22 case report, 20 case series, 3 case-control and 27 cross-sectional studies) that met the inclusion criteria were included in the final analysis ( Figure 1 ). Of the selected publications, 51 were written in English while 21 were written in Chinese. The general information on eligible publications can be found in Table 1 February 14, 2020 . Quality assessment of the literatures showed that cross-sectional studies achieved a good quality (median score, 7.0) while case report/series were fair (median score, 6.0). A total of 3,470 COVID-19 patients were included, of which 3,468 confirmed cases were based on the positive SARS-CoV-2 on RT-PCR assay while two cases 25 were diagnosed according to coronavirus antibody detections. The age of the COVID-19 patients ranged from 17 days to 92 years old. Male gender accounted for 1822 (52.6%) of the selected patients. Among 1,573 patients who were asked for smoking status, 208 (13.2%) were self-reported smokers. Through a synthesized analysis, a total of 2182 cases (76.0%) had a history of transmission exposures, i.e., the patients were either Wuhan residents or travelled to Wuhan within the past 14 days. The most common symptom of COVID- 19 Table 3 for details). author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the .  Collectively, a total of 185 [11.5%] patients were admitted to intensive care unit (ICU) while the overall CFR was 3.7% ( Table 2) . Our further analysis showed that COVID-19 patients admitted in Hubei province (Wuhan in particular) suffered from a significant higher ICU admission rate than that outside of Hubei, China (21.9% vs. 2.5%, p<0.001). Also, CFR attributed to COVID-19 in Hubei province, China was significantly higher than that in non-Hubei, China (10.4% vs. 0.6%, p<0.001). author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . indicating that COVID-19 may not have a gender predisposition. Although 13.2% of the selected patients had a history of smoking, we could not draw the conclusion that smokers are less susceptible to the viral infection than non-smokers, as the patients included in our analysis were from a sample pool that was not representative the general population by smoking status. This systematic review showed that most of the patients (76.0%) had a history of Wuhan-related exposures. This is of great importance for early quarantine for the subjects with emerging infectious disease when specific treatments are not available. Actually, we have learned from the SARS outbreak 17 years ago that early identification, early isolation and early management would lead to the stop of viral transmissions from human to human 79 . Collectively, there were 36% of the COVID-19 patients absent from fever as the onset symptom. In this case, such patients may have been ignored at the early stage if we focused heavily on fever examination for initial screening. Our composite analysis showed that fever remains the most common symptom (83.0%) in patients with COVID-19. However, the proportion of fever is somewhat lower than that of other coronavirus related respiratory illness, such as SARS (100%) 80 81 or middle east respiratory syndrome (MERS) (98%) 82 . Similarly, the accompanied symptoms of dyspnea (14.5%) and diarrhea (6.1%) are relatively less common in patients with COVID-19 than those seen in SARS and MERS 52 . More importantly, this study revealed an overall CFR of 3.7%, which was quite similar to that reported by the WHO official statistics as of March 5, 2020 (CFR 3.7%, 3,015 died of 80,565 cases) 4 . Nevertheless, the CFR of COVID-19 was much lower than that of SARS (9.6%) 83 and MERS (37.1%) 84 . By far, the mechanisms underlying the varying symptoms and CFR for these three coronavirus-infected diseases are not fully understood. One reason may be that there were still some COVID-19 patients being treated in hospitals at the time of the manuscripts submitted, so the outcome (death or recovery) is not known yet. Additionally, we suppose that the varied tropism as well as virulence of these three coronaviruses may in part account for the discrepancies, which warrants further All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . author/funder, who has granted medRxiv a license to display the preprint in perpetuity. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . author/funder, who has granted medRxiv a license to display the preprint in perpetuity. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . author/funder, who has granted medRxiv a license to display the preprint in perpetuity.

In [ ]:
tfDTM['sum']=tfDTM['diagnos']+tfDTM['mechan']+tfDTM['treatment']+tfDTM['coronavirus']+tfDTM['dissemin']\
                + tfDTM['medic']+tfDTM['symptom']

so=tfDTM.sort_values(by='sum',ascending=False).index
d=[]
c=[]
t=[]
for k in so[:5]:
    d.append(daaw['body'].iloc[k])
    c.append(daaw['abstracts'].iloc[k])
    t.append(daaw['title'].iloc[k])
print(len(' '.join(c)))


In [ ]:
for i in range(0,3):
    print(str(i+1)+'.',t[i],'\n')
    print('Abstract:',c[i],'\n')
    print('Body:',d[i][:10000])
    print('\n')

## Research reports on Diagnosis and therapeutics to track variations over time

1. Study of the mental health status of medical personnel dealing with new coronavirus pneumonia 

Abstract: Objective: This paper studied the relationship between personality traits and mental health conditions of medical personnel to provide a basis and reference for the implementation of targeted education on mental health. A self-report inventory, the Symptom Checklist-90 (SCL-90), was used to investigate the mental health status of 548 medical personnel dealing with the new coronavirus pneumonia in eight provinces and cities of China. The overall mean SCL-90 score and mean values of factors (somatization, obsessive-compulsive, anxiety, phobic anxiety, and psychoticism) of the medical personnel were significantly higher than in the norm group (p < 0.05), while their average interpersonal sensitivity score was significantly lower (p < 0.01). In addition, personal factors affecting the mental health status of medical personnel were identified. ( all p < 0.05). The overall mental health status of medical personnel responding to new coronavirus pneumonia is generally higher than that of the norm group in China. The results of this study should contribute to measures to alleviate the psychological pressures on medical personnel dealing with the new coronavirus epidemic in China. [Keywords] medical personnel; new coronavirus pneumonia; SCL-90 inventory; mental health; influencing factors All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. 

Body: Novel coronavirus pneumonia (NCP) is a pathogenic coronavirus often referred to as the novel coronavirus. On January 12, 2020, WHO officially named the disease Coronavirus Disease 2019 (COVID-19). The first case of COVID-19 was reported in Wuhan in China on December 12, 2019 [1] [2] as causing severe acute respiratory infection (SARI). Coronaviruses are a large family of viruses known to cause illnesses such as cold and more serious diseases such as Middle East respiratory syndrome (MERS) and severe acute respiratory syndrome (SARS). COVID-19 is a new strain of coronavirus that has never been found in the human body before [3] [4] . COVID-19 patients typically show symptoms such as fever, coughing, shortness of breath, and difficulty in breathing. In more severe cases, the disease can lead to pneumonia, severe acute respiratory failure, kidney failure, and even death [5] . There is as yet no specific treatment for COVID-19. Patients mainly receive symptomatic treatment and care for the prevention of complications, while supplementary medical services also appear to be very effective for infected people. The rate of COVID-19 infections has increased rapidly in a short period of time. As of February 17, 2020, the number of confirmed cases exceeded 70,000, with 1,770 deaths [6] . In addition, the disease can cause secondary infections, which has created a huge burden and pressure for the prevention and treatment of the disease in various places. At present, more than 30,000 medical personnel from various medical teams across the country have provided support in Wuhan [7] . Though the help from these medical personnel has relieved the local pressure for medical care to save critically ill COVID-19 patients, there have been serious infections among the medical staff in Wuhan and other places in Hubei Province. At present, over 3000 medical personnel have been infected, which has greatly increased the psychological pressure they experience. In the face of the catastrophic health emergency of COVID-19, medical personnel have been affected by different kinds of subjective and objective factors and confront several mental health problems. Mental illness is a form of human stress response, an explanatory, emotional, and defensive response within the human body, and a physiological response of the human body to the impact of needs or injuries. Therefore, this study aims to analyze the psychological state of medical personnel dealing with COVID-19 and its influencing factors in order to provide an objective basis for the prevention of further transmission, interventions, and countermeasures for COVID-19. This study adopted convenience sampling to recruit research subjects. From January 25 to February 16, 2020, 560 medical personnel from 12 hospitals in eight provinces and cities across the country were enrolled as research subjects. or emergency department, level of concern whether you and your family have been infected, degree of suspicion that you were infected when coronavirus-related symptoms occurred, and whether you have received medical observation recently. The SCL-90 self-report inventory: The SCL-90 inventory, compiled by Derogatis in 1975, includes 10 factors and a total of 90 items. Each factor reflects the symptoms and pain of a patient in a certain aspect, and the distribution of symptoms can be understood through the factor scores. The 10 factors include somatization, obsessive-compulsive, depression, anxiety, hostility, phobic anxiety, paranoid ideation, psychoticism, sleep, and diet. Each item was scored using a 5-point Likert scale, ranging none, mild, moderate, moderate to severe, to severe. The total score is the sum of the scores of the 90 items. In previous studies, the homogeneity reliability of the total SCL-90 scale was 0.97, and the homogeneity reliability of each sub-scale was also greater than 0.68. The test-retest reliability was greater than 0.7, and the content validity was above 0.85, which suggest sound reliability and validity. The study used online questionnaires for data collection. Researchers conducted surveys upon the completion of general training. All procedures were approved by the Ethics Committee of Harbin Medical University(HRBYKD-A26). The research purpose and methods were explained to subjects to seek their cooperation. Online informed consent forms were signed by participants. They were informed that their participation was completely voluntary, and they could withdraw from the study at any time. The link of the online questionnaire was then sent to a total of 560 medical personnel via the Internet. (1) In this study, data analysis was performed after logical checks using the statistical software SPSS 22.0. A p-value less than 0.05 (p < 0.05) was considered statistically significant. (2) Statistical description: The mean value, standard deviation, and frequency were used to describe the demographic data of medical personnel, while the mean value and standard deviation were used to describe the scores for the mental health status of medical personnel responding to COVID-19. (3) Statistical inference: Multivariate linear regression was adopted to analyze the impact of the demographic data of medical personnel on their mental health status. A total of 560 questionnaires were distributed, and 548 valid questionnaires were recovered, for an effective recovery rate of 97.90%. These 548 questionnaires were completed by medical personnel from eight provinces and cities in China, namely Heilongjiang, Liaoning, Jilin, Inner Mongolia, Tianjin, Sichuan, Shanxi, and Shandong. Details of the respondents' personal information are given in Table 1 . author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the  All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the author/funder, who has granted medRxiv a license to display the preprint in perpetuity. 2.2 SCL-90 factor scores of medical personnel compared with national norms [8] The overall average of SCL-90 and mean values of factors (somatization, obsessive-compulsive, anxiety, phobic anxiety, and psychoticism) of medical personnel were significantly higher than that of the norm group (p < 0.05 or p < 0.01), while the average score of the interpersonal sensitivity factor of medical personnel was significantly lower than that of the norm group (p < 0.01). Details are provided in Table 2 . author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . https://doi.org/10.1101/2020.03.04.20030973 doi: medRxiv preprint Stepwise linear regression was performed using the total score of mental health status as the dependent variable and 17 items of personal information as independent variables. The 17 items include: hospital, department, occupation, gender, age, highest education level, work experience, level of expertise, marital status, any children, living status, whether you have participated in training for handling of public health emergencies, whether family members support your working on the front line against coronavirus, whether you have supported in affected areas in Hubei, designated hospitals, department of infectious diseases, fever clinics or emergency department, level of concern whether you and your family have been infected, degree of suspicion that you were infected when coronavirus-related symptoms occurred, and whether you have received medical observation recently. Values were assigned to independent variables using the method shown in Table 3 . The α -values for importing and exporting a variable in the regression equation were set to 0.10 and 0.15, respectively. Factors affecting the mental health and status of medical personnel based on their significance from high to low are: the degree of suspicion that they were infected when the novel coronavirus-related symptoms occurred, the level of concern whether they and their family members have been infected, age, whether they have supported in affected areas in Hubei Province, designated hospitals, and other places for the novel coronavirus, and whether family members support them working on the front line (p < 0.05). Details of the regression results are listed in Table 4 . author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . https://doi.org/10.1101/2020.03.04.20030973 doi: medRxiv preprint COVID-19 is a fulminant infectious disease. As it is highly contagious, many people are frightened by it and even talk fearfully about coronavirus, which can also be observed in front-line medical staff. The results of the study have shown that the overall mean of the SCL-90 and the mean values of the factors (somatization, obsessive-compulsive, anxiety, phobic anxiety, and psychoticism) of medical personnel were significantly higher than that of the norm group (p < 0.05), while the average score of the interpersonal sensitivity factor of medical personnel was significantly lower than that of the norm group (p < 0.01). More specifically, medical personnel are most of the people closest to COVID-19 patients, so they are at high risk of exposure to the disease. Moreover, they have a deep understanding of the dangers of COVID-19, so they are prone to anxiety and fear. The infection protection procedures for COVID-19 are highly complex and medical staff need to constantly change clothes and replace protective equipment, so they are more likely to establish obsessive-compulsive behaviors. Medical personnel, especially young medical staff, have less experience in the field and in dealing with difficulties and hardshi


2. Evaluation of the clinical characteristics of suspected or confirmed cases of COVID-19 during home care with isolation: A new retrospective analysis based on O2O 

Abstract: The recent outbreak of the novel coronavirus in December 2019 has activated top-level response nationwide. We developed a new treatment model based on the online-to-offline (O2O) model for the home isolated patients, because in the early stages the medical staff were insufficient to cope with so many patients. In this single-centered, retrospective study, we enrolled 48 confirmed/suspected COVID-19 patients who underwent home isolation in Wuhan between January 6 and January 31, 2020. By WeChat and online document editing all patients were treated with medical observation scale. The clinical indications such as Fever, Muscle soreness, Dyspnea and Lack of strength were collected with this system led by medical staff in management, medicine, nursing, rehabilitation and psychology. The mean age of 48 patients was 39·08±13·88 years, 35(72·9%) were women. Compared with non-hospitalized patients, inpatients were older(≥70years, 2·4% vs 33·3%, P<0·04). All inpatients had fever, 50% inpatients had coughs and showed infiltration in both lungs at the time of diagnosis. 33·3% All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. : medRxiv preprint inpatients exhibited negative changes in their CT results at initial diagnosis. The body temperature of non-hospitalized patients with mild symptoms returned to normal by day 4-5. While dyspnea peaked on day 6 for non-hospitalized patients with mild symptoms, it persisted in hospitalized patients and exacerbated over time. The lack of strength and muscle soreness were both back to normal by day 4 for non-hospitalized patients. Monitoring the trends of symptoms is more important for identifying severe cases. Excessive laboratory data and physical examination are not necessary for the evaluation of patients with mild symptoms. The system we developed is the first to convert the subjective symptoms of patients into objective scores. This type of O2O, subjective-to-objective strategy may be used in regions with similar highly infectious diseases to minimize the possibility of infection among medical staff. All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. 

Body: The recent outbreak of the novel coronavirus in December 2019 (COVID-19) has activated top-level response nationwide and has been classified as a public health emergency of international concern (PHIEC) by the World Health Organization (WHO). 1 By 24:00 on February 16, 2020, 70,548 confirmed cases, 10,644 severe cases, 1,770 deaths, and 546,016 close contact cases have been identified in 31 provinces (autonomous regions and municipalities) and the Xinjiang Production and Construction Corps of China. 2 The SARS-CoV-2-induced pneumonia has rapidly spread from Wuhan to 21 other countries, including the United States, Japan, Italy and Germany 3, 4 , demonstrating high levels of infectivity and pathogenicity. [5] [6] [7] The fever clinic of Tongji hospital in Wuhan has been the center of this outbreak. During the early phase of this outbreak, a large number of patients poured into the fever clinic, which far exceeded the medical resources that the hospital could equip. The medical staff were obviously insufficient to cope with it, and were prone to a wide range of cross-infection between doctors and patients. Based on this situation many patients had to be quarantined at home due to objective reasons and could not receive effective medical guidance. Therefore, we developed a new treatment method based on the online-to-offline (O2O) business model 8 for close contact, suspected (currently known as clinical diagnosis) and confirmed patients that were under quarantine. We developed a medical observation scale according to the patients' first symptoms and new symptoms which can be filled out by the patients on their smartphones or computers. Our online multidisciplinary team (medicine, rehabilitation, psychology and nursing) can then provide guidance and advice for patients based on the subjective changes in their symptoms. This method ensures that the patients follow an orderly treatment-seeking strategy that begins from their home and extends to the community and finally to the hospital. Our strategy not only helps relief the problem of scarce medical resources and reduce unnecessary cross-infection in hospitals, but it also increases people's self-management ability and cooperation and encourages them to participate in health monitoring. In addition, in accordance with the guidelines of home care for isolation patients issued by WHO 9 , the infection of family members was not increased during the strict self-isolation at home in this study. In this study, we reviewed the observational data of isolated patients and examined the temporal relationship among their clinical symptoms in order to help physicians in areas with insufficient medical resources to effectively identify and treat critical patients. Evidence before this study Since December 2019, COVID-19 has exploded in Wuhan, China. So far, it has spread all over the world. At the initial stage of the outbreak of this new type of respiratory infectious disease, due to the obvious shortage of medical resources, a large number of patients had to be isolated and treated at home. Up to Feb 24, 2020, we have searched PubMed for all the literatures related to COVID-19. There is no description of the clinical characteristics and outcome of patients during home isolation. In addition, we have created a medical observation scale designed for patients quarantined at home. Combined with the O2O model the patients were evaluated and treated. We report the clinical characteristics and outcome of 48 confirmed/suspected COVID-19 patients from 188 individuals who participated in the medical observation during home care and isolation. Compared with non-hospitalized patients, inpatients were older(≥70years, 2·4% vs 33·3%, P<0·04). All inpatients had fever, 50% inpatients had coughs and showed infiltration in both lungs by at the time of diagnosis. 33·3% inpatients exhibited negative changes in their CT results at initial diagnosis. The body temperature of non-hospitalized patients with mild symptoms returned to normal by day 4-5. While dyspnea peaked on day 6 for non-hospitalized patients with mild symptoms, it persisted in hospitalized patients and exacerbated over time. The lack of strength and muscle soreness were both back to normal by day 4 for non-hospitalized patients. Monitoring the trends of symptoms is more important for identifying severe cases. Excessive laboratory data and physical examination are not necessary for the evaluation of patients with mild symptoms. The system we developed is the first to convert the subjective symptoms of patients into objective scores. This type of O2O, subjective-to-objective strategy may prove useful in areas with insufficient medical resources to minimize the possibility of infection among patients and medical staff. This is a single-center retrospective study of individuals who came into close contacts with COVID-19 patients in Tongji hospital of Wuhan between January 6 and January 31, 2020. The study was approved by Tongji Hospital Ethics Committee before data were collected retrospectively. Among the 188 individuals who participated in the medical observation, 114 were excluded from this study due to no signs of discomfort during the 14-day quarantine (108 cases) and non-COVID-19 (6 cases). Patients who were confirmed with or suspected of COVID-19 and were willing to undergo medical observation. Subjects that meet any one epidemiological history or meet two clinical manifestations without epidemiological history were defined as suspected cases. Confirmed case is defined as a suspected case with one of the following etiological evidence: Positive for SARS-CoV-2 nucleic acid in respiratory or blood samples detected by RT-PCR; Virus sequence detected in respiratory or blood samples shares high homology with the known sequence of SARS-CoV-2. (1) Pregnant or breast-feeding women; (2) <18 or >75 years old; (3) Unable to cooperate with data reporting. (1) The end date of the observation was January 31, 2020; (2) When the patient was admitted to the hospital or died; (3) When the patient was clinically cured: with normal CT imaging and at least twice of SARS-CoV-2 nucleic acid assays were negative. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . https://doi.org/10.1101/2020.02.26.20028084 doi: medRxiv preprint (5) Dyspnea: 1 = Not troubled by breathlessness except with strenuous exercise; 2 = Troubled by shortness of breath when hurrying on a level surface or walking up a slight hill; 3 = Walks slower than normal based on age on a level surface due to breathlessness or has to stop for breath when walking on level surface at own pace; 4 = Stops for breath after walking 100 meters or after a few minutes on a level surface; 5 = Too breathless to leave the house (6) Lack of strength: 1 = No lack of strength; 2 = Mild: Slightly lacked strength, able to do physical work, improves after rest but does not recover to normal; 3 = Moderate: Lacked strength, feels weak, able to persist in daily activities and work but light physical work is very tiring and does not recover to normal after long periods of rest; 4 = Severe: Extremely lacked strength, unable to conduct normal activities, feels tired at rest, cannot talk  We first obtained the corresponding symptoms from the 6 subjects under observation, then continuously added new symptoms to the online scale as we furthered our understanding of the disease. We graded the symptoms based on our clinical work experience in lay language and compiled them into the medical observation scale. We gave our scale to 34 experts for evaluation of the items and score criteria. Two rounds of expert consultations were conducted with 17 experts from the emergency department, respiratory department, intensive care unit, and infectious disease department each round. Statistical analysis of the consultation results showed that the response rates of the two rounds of expert consultation were 100% and 88·24%, and the mean authoritative coefficient was 0·855. The degree of coordination of expert opinions (W value) was 0·204 and 0·293 for the two rounds, respectively, and the W values were statistically significant (P<0·01). The levels of enthusiasm and authority, concentration of opinion, and consistency in item evaluation results were relatively high among the experts, which demonstrates that the "Quarantine Management Assessment Form for Suspected or Confirmed COVID-19 Patients with Mild Symptoms" we designed was scientifically sound and reliable and can be used for the centralized quarantine, observation and care of suspected or confirmed cases. This assessment form includes general demographics, past history, current medical history, current symptoms and signs, and diagnostic recommendations consisting of 5 primary indicators, 22 secondary indicators, and 83 tertiary indicators. Empirical therapy consisted of oral moxifloxacin or levofloxacin (consider tolerance) and arbidol. Arbidol was approved in China and Russia for influenza treatment. In vitro studies showed that arbidol had inhibitory effects on SARS. 11 Statistical analysis was performed using SPSS26·0. Normally distributed measured data were expressed as mean±standard deviation (x±s) and compared using the t-test based on homogeneity of All rights reserved. No reuse allowed without permission. author/funder, who has granted medRxiv a license to display the preprint in perpetuity. The copyright holder for this preprint (which was not peer-reviewed) is the . https://doi.org/10.1101/2020.02.26.20028084 doi: medRxiv preprint variance. Categorical data were expressed as frequency and percentage and compared using the P<0·05 was considered statistically significant. Demographics, baseline characteristics, and clinical outcomes of 48 COVID-19 patients quarantined at home ( Table 1 ). The mean age of the 48 patients was 39·08±13·88 years. The patients 


3. Direct Measurement of Rates of Asymptomatic Infection and Clinical Care- Seeking for Seasonal Coronavirus 

Abstract: The pandemic potential of the novel coronavirus (nCoV) that emerged in Wuhan, China, during December 2019 is strongly tied to the number and contagiousness of undocumented human infections. Here we present findings from a proactive longitudinal sampling study of acute viral respiratory infections that documents rates of asymptomatic infection and clinical care seeking for seasonal coronavirus. We find that the majority of infections are asymptomatic by most symptom definitions and that only 4% of individuals experiencing a seasonal coronavirus infection episode sought medical care for their symptoms. These numbers indicate that a very high percentage of seasonal coronavirus infections are undocumented and provide a reference for understanding the spread of the emergent nCoV. . CC-BY 4.0 International license It is made available under a author/funder, who has granted medRxiv a license to display the preprint in perpetuity. is the (which was not peer-reviewed) The copyright holder for this preprint . 

Body: The Wuhan novel coronavirus (nCoV) appears to have emerged in humans in the Hubei province of China during December 2019 (1) . The virus was definitively identified in the first week of 2020 and as of January 29 had spread to 15 other countries; 6065 confirmed cases, 9239 suspected cases, and 132 deaths have been reported (2) ; however, the total number of infections (confirmed, suspected and unreported) remains unknown. Quantification of the reporting rate, i.e. the ratio of confirmed and suspected cases to total infections, is vital for delineation of the prevalence, scope and potential spread of nCoV. Indeed, the speed and geographic extent of its translocation throughout China and internationally are likely tied to the true prevalence of the pathogen and suggest that there are many more infections (i.e. the reporting rate is low), that contagiousness is not negligible among these undetected infected persons, and that nCoV will be difficult to contain. To contextualize the issue of unreported infection, we here present the findings from a recent proactive sampling project carried out in New York City (NYC) that documented rates of asymptomatic infection and clinical care-seeking among individuals shedding seasonal CoV (types: HKU1, 229E, NL63 and OC43). Data are derived from sampling performed between October 2016 and April 2018 as part of the Virome project, a proactive sampling of respiratory virus infection rates, associated symptom reports and rates of seeking clinical care. We enrolled 214 healthy individuals from multiple locations in the Manhattan borough of NYC. Cohort composition is described in Galanti et al. (3, 4) and includes: children attending two daycares, along with their siblings and parents; teenagers and teachers from a high school; adults working at two emergency departments (a pediatric and an adult hospital); and adults working at a university medical center. The cohort was obtained using convenience sampling, and all participants were younger than 65 years. While the study period spanned two years from October 2016 to April 2018, some individuals enrolled for a single cold and flu season (October -April) and others for the entire study period. Participants (or their guardians, if minors) provided informed consent after reading a detailed description of the study (CUMC IRB AAAQ4358). Nasopharyngeal samples were collected by study coordinators once a week irrespective of participant symptoms. Samples were screened using the GenMark eSensor RVP system for 18 different respiratory viruses, including coronavirus 229E, NL63, OC43, and HKU1. Sample collection and extraction followed the same protocol as in Shaman et al. (5) . In addition, participants completed daily self-reports rating nine respiratory illnessrelated symptoms (fever, chills, muscle pain, watery eyes, runny nose, sneezing, sore throat, cough, chest pain) each of which was recorded on a Likert scale (0=none, 1=mild, 2=moderate, 3=severe). The daily self-report also documented whether participants had sought medical attention, stayed home or taken cold and flu-related medications (both over-the-counter and prescription) as a consequence of their listed symptoms (see (4) for further survey details). Table 1 . Definitions of symptomatic infections. All symptom definitions are described in reference to a -3 to +7-day window around the date of the initial positive swab for an infection episode. Note, Definition 4 is relative to an individual's long-term average total symptom score. Definition 1 At least one day with a daily score >3 Definition 2 Minimum two individual symptoms >0 and at least one symptom >1 Definition 3 Total symptom score >9 Definition 4 Total symptom score greater than twice the weekly average for the infected individual Definition 5 Total symptom score >0 (i.e. any reported symptom) We defined an infection (or viral) episode as a group of consecutive weekly specimens from a given individual that were positive for the same virus (allowing for a one-week gap to account for false negatives and temporary low shedding). Medically attended illness, sick days at home and medicine uptake were defined as episodes in which the participants reported seeking care, staying home or taking medicines due to respiratory symptoms. We classified all infection episodes as symptomatic or asymptomatic according to individual symptom scores in the days surrounding the date of the first positive swab of an episode. We used multiple definitions as a standard for symptomatic infection does not exist (Table 1) . These symptom definitions are described in reference . CC-BY 4.0 International license It is made available under a author/funder, who has granted medRxiv a license to display the preprint in perpetuity. is the (which was not peer-reviewed) The copyright holder for this preprint . to a -3 to +7-day window around the date of the initial positive swab for an infection episode. Daily score is defined as the sum of the 9 individual symptoms (range: 0-27). Total symptom score is the daily symptom score summed over the -3 to +7-day window. Table 2 presents the findings for seasonal CoV from this proactive sampling effort. 135 CoV infection episodes were identified. CoV-OC43 infection was most frequent (n=61), and CoV-NL63 infection was least frequent (n=15). Note that co-infections occurred; hence, the sum of the type infections is greater than the total reported in the final column of Table 2 . Adults (20+ years old) constituted 64% of the cohort, provided 67% of collected swab specimens, and experienced 50% of all CoV infection episodes. Based on Definitions 1-4, the majority of CoV infection episodes were asymptomatic. When stratified by type, differences emerge: CoV-NL63 was more than twice as likely to produce symptomatic infection episodes as CoV-229E (by Definitions 1-4). Definition 5, which defines symptomatic infection as the manifestation of any symptom during the 11day window spanning the initial positive swab collection, indicates that a majority of infected persons reported at least one mild cold-related symptom. Participants reported taking medicine in response to cold and flu symptoms for 26% (n=35) of CoV infection episodes. Participants were three times more likely to take medicine in association with CoV-NL63 infection (40%) than CoV-229E infection (12%). 10% of participants reported staying home from work or school during CoV infection episodes, and 4% of participants reported seeking medical care.  Reporting of seasonal CoV prevalence has been limited to date. The US CDC only began releasing CoV positivity rates in 2018, and understanding of its epidemiological features is incomplete. Here, we present direct measurement of rates of seeking clinical care and show that only 4% (6 of 135) of persons shedding seasonal CoV sought care. Reporting rates and rates of seeking medical care may differ, as most individuals visiting a clinician for acute respiratory illness are not tested for seasonal CoV. However, testing for the 2019 nCoV is expected to be more robust than for seasonal CoV as there is pressing need to understand and contain this novel pathogen. This high testing rate could result in the reporting rate for nCoV equaling the rate of seeking medical care. Thus, if rates of seeking medical care for nCoV were to align with seasonal CoV, the reporting rate for nCoV would approach 4%. Should the remaining 96% of unreported infections be sufficiently contagious, their high numbers could support rapid spread of nCoV. It is important to note that rates of seeking medical care for nCoV may not align with the rates found here for seasonal CoV: concern over the virus may lead to elevated rates of seeking medical care; our estimates for seasonal CoV represent a moderate sample size; and the symptom severity of emergent Wuhan nCoV may differ from seasonal CoV. Further, early outbreak instances of death due to nCoV occurred predominantly in elderly patients, and often elderly with co-morbidities (6) . The data we present here for seasonal CoV are from a cohort with no individuals 65 year or older, so the findings identified here might not have bearing on the elderly. Still, several recent studies suggest that reporting rates for nCoV may align with the rates found here for seasonal CoV. Chinazzi et al., (7) used an agent-based modeling approach to estimate the overall nCoV incidence (reported and unreported) necessary to support the observed numbers of geographically exported infected individuals as of January 21, 2020. They estimated a 5-16% nCoV reporting rate depending on model assumptions. In addition, phylogenetic analysis has identified Wuhan nCoV as a betacoronavirus, the genus that includes CoV-OC43 and CoV-HKU1, as well as CoV-SARS and CoV-MERS (8) . Among our cohort participants, 7% (6 of 91) of infection episodes with seasonal betacoronavirus (OC43 and HKU1) sought medical care. . CC-BY 4.0 International license It is made available under a author/funder, who has granted medRxiv a license to display the preprint in perpetuity. is the (which was not peer-reviewed) The copyright holder for this preprint . The impact of unrecognized and unattended infections on transmission dynamics is likely a key determinant of whether control of the nCoV outbreak is possible. For example, although cases of seroconversion without sign of disease were documented during the SARS outbreak, asymptomatic and subclinical rates are believed to have been low. No cases of asymptomatic SARS transmission were documented (9) , which may have facilitated the containment of the epidemic. A higher asymptomatic ratio was reported for MERS infections, which presented with a broader spectrum of clinical manifestations: 21% of laboratory identified cases were mild or asymptomatic and 48% had severe disease or died (10). The potential for asymptomatic transmission of MERS has been reported (11);


In [ ]:
tfDTM['sum']=tfDTM['social']+tfDTM['spread']+tfDTM['econom']+tfDTM['covid']+tfDTM['wildlif']\
                +tfDTM['transmiss']+tfDTM['anim']+tfDTM['geographi']+tfDTM['demographi']

so=tfDTM.sort_values(by='sum',ascending=False).index
d=[]
c=[]
t=[]
for k in so[:5]:
    d.append(daaw['body'].iloc[k])
    c.append(daaw['abstracts'].iloc[k])
    t.append(daaw['title'].iloc[k])
print(len(' '.join(c)))


In [ ]:
for i in range(0,3):
    print(str(i+1)+'.',t[i],'\n')
    print('Abstract:',c[i],'\n')
    print('Body:',d[i][:10000])
    print('\n')

## Transmission from Animal Hosts(s)

1. Emerging Diseases at the Interface of People, Domestic Animals, and Wildlife. The Role of Wildlife in our Understanding of Highly Pathogenic Avian Influenza 

Abstract:  

Body: The Wildlife Conservation Society (WCS) operates five parks in New York City, including the zoos at Central Park, Queens, and Prospect Park, the New York Aquarium and the Bronx Zoo. In addition, WCS manages some 400 conservation projects in 60 countries around the world. The commitment to the well-being of animals in zoos and to the conservation of critical landscapes around the globe provides a unique and robust set of competencies to provide critical health care programs that work to ensure the health of people, domestic animals, and wildlife. This was clearly demonstrated in 1999, when veterinary pathologists of the WCS were the first to connect the deaths of people, wild free-ranging birds, and zoo birds to a new disease to the Western Hemisphere. With samples from the WCS wildlife health surveillance program, the disease was confirmed to be West Nile virus. The Field Veterinary Program (FVP) of the Wildlife Health Sciences Unit of WCS is active on four continents and performs community-based wildlife population health monitoring and surveillance. This on-the-ground commitment to assessing the long-term health of wild populations provides critical information that can serve as an early warning system for the emergence of new and renewed pathogens at the rural wildlife interface as well as the regional urban marketplace. It has been WCS FVP teams that provided the observations and samples that confirmed that gorillas were dying of Ebola virus in central and west Africa. During recent years, the broad disease surveillance techniques of the FVP have provided a baseline of information on the health of mammals, birds, and reptiles in various parts of the world. Low pathogen avian influenza (LPAI) is endemic in wild migratory waterfowl populations. Best theories suggest that LPAI H5N1 moved from wild birds to domestic fowl, and in 1997, highly pathogenic avian influenza (HPAI) H5N1 was first identified as a disease of domestic fowl that spread to people in close contact with them in Hong Kong. The resurgence of this disease in Southeast Asia in 2004 brought a new wave of poultry and human deaths. One must study the cultural and agricultural practices in Asia to understand how this disease might mutate to a highly pathogenic form in domestic fowl, how it might spread to humans, and how it might jump back to wild migratory waterfowl. Religious practices, such as merit bird releases, bring people and passerine birds into intimate contact. The popular practice of cockfighting results in people moving these birds across vast regions and caring for injured cocks, which then results in human contact with bird blood and body fluids. It is common practice for people in rural areas to live in close proximity with their domestic animals, often times sharing room in the home or nearby. It is a typical scene to see cattle, ducks, chickens, and pigs living in or very near a family dwelling. The methods in which ducks are raised can include providing access to recently harvested rice paddies, where mixing with wild birds is commonplace. The aquatic environment is an excellent media for the persistence of viable avian influenza virus. The trade in wild animals is a multibillion dollar global operation. The animal markets or "wet markets" of Asia are a mixing bowl of domestic animals, wildlife from near and far, and people. Most often, sanitation and hygiene are very poor to nonexistent, and both people and animals are under a tremendous amount of stress, lowering immuno-competency. Those in the marketplace are handling live birds and butchering others without any personal protection and often live, eat, and sleep in their shops amongst their animals for sale. This serves as an excellent environment in which pathogens can mutate and jump into novel species. While it is uncertain that civet cats were the source of severe acute respiratory syndrome (SARS) in the Guangdong markets of China, it is clear they carried the disease. Since initial findings implicated civet cats, other species, such as domestic cats and fruit bats, have been shown to harbor the virus. The FVP has been performing surveillance of wild bird populations for some 15 years in selected landscapes around the globe. Our work has demonstrated low pathogenic strains of avian influenza as far south as the Falkland Islands. In August 2005, a multi-disciplinary team comprised of FVP field veterinarians and Mongolian scientists performed field surveillance research in nine different sites along the migratory waterfowl flyway in Mongolia, including Erhel Lake, where a bird die-off was reported. At these nine sites, 850 fecal samples were collected, including 433 samples at Erhel Lake. All birds sampled were identified as to species and this information recorded. All species at a given location were identified and individuals counted so prevalence rates could be calculated. No virus was detected in live bird fecal culture samples. Six dead birds were necropsied. HPAI H5N1 was isolated from a whooper swan on Erhel Lake. The organism was isolated from samples provided to the United States Department of Agriculture ARS Poultry Laboratory in Athens, Georgia. The H5N1 isolate was determined to be a Clade II strain and selected by the World Health Organization and Centers for Disease Control and Prevention to be included in the research efforts to produce a human vaccine. The results of this work suggest that the HPAI H5N1 has entered the migratory waterfowl flyway. The fact that the vast majority of birds appeared healthy and their fecal cultures were negative also suggests that the prevalence of virus is low. Further, the observation that sick birds died and one of these few animals had H5N1 raises the question as to whether the migratory birds in this surveillance project were highly competent carriers of the organism. Further work must be performed to determine if there is a species of wild bird that can carry the disease for long distances. If wild birds prove to die precipitously from the disease, then wildlife birds are not likely the source of spread for HPAI H5N1, but rather one must consider other sources for spread into domestic poultry populations. These sources could easily be domestic poultry or the vast number of animals moving through the trade in domestic animals and wildlife for meat and pets that are being placed into markets in distant and disparate sites. The most effective method of protecting people from contracting HPAI H5N1 is to educate them on proper sanitation and hygiene for themselves and their domestic poultry. Basic preventive methods such as hand washing and proper protective garb when handling poultry will greatly reduce the risk of spread to humans from animals. Killing wild birds will be an unsuccessful mitigation method. It will not halt the spread of this disease in domestic poultry and subsequently into those that are in close proximity to these animals. It is most important to note that H5N1 is but one of many possible strains of avian influenza that might someday become a competent pathogen of people. In addition, avian influenza is but one of a plethora of diseases that might make the evolutionary jump from being an animal pathogen to becoming a human disease. We must begin to think more holistically about our approach to the health of people and animals. We must consider that all living things on this world are intimately linked and only by understanding the rela-tionships can we begin to provide sound solutions to complex disease threats. Halting the trade in wildlife is a critically important step to lowering the risk of pathogens emerging from wildlife to domestic animals and people and will slow the loss of biodiversity on our planet. In recent years, outbreaks of diseases such as avian flu, severe acute respiratory syndrome (SARS), the Ebola virus, and mad cow disease have frightened the public, disrupted global commerce, caused massive economic losses, and jeopardized diplomatic relations. These diseases have also shared a worrisome key characteristic: The ability to cross the Darwinian divide between animals and people. None of these illnesses depends on human hosts for its survival; as a result, they all persist today, far beyond the reach of medical intervention. Meanwhile, humanity has become vulnerable to cross-species illnesses, thanks to modern advances such as the rapid transportation of both goods and people, increasing population density around the globe, and a growing dependence on intensified livestock production for food. The global transport of animals and animal products, which includes hundreds of species of wildlife, also provides safe passage for the harmful bacteria, viruses, and fungi they carry, not to mention the prion proteins that cause insidious illnesses such as mad cow disease and chronic wasting disease in deer and elk. Adding to the risks is the fact that while many people in the developed world would scarcely recognize meat if it did not come wrapped in clear plastic, the vast majority of people on the planet today still slaughter animals for meat themselves or buy it fresh, salted, or smoked in open-air markets. These markets generally go uninspected by health officials, and consumers rarely have access to good health care, education on hygiene, common vaccines, or antibiotics. Not only is local and national health care often a problem, internationally no agency is responsible for, or capable of, monitoring and preventing the myriad diseases that can now cross the borders between countries and species. More specifically, no organization has the mandate to pursue policies based on a simple but critically important concept: That the health of people, animals, and the environment in which we all live are inextricably linked. Thus, for example, the U.S. Department of Agriculture works to protect only the United States livestock industry and has scaled back the attention it pays to animals outside the U


2.  

Abstract: Zoonoses with a wildlife reservoir represent a major public health problem, affecting all continents. Hundreds of pathogens and many different transmission modes are involved, and many factors influence the epidemiology of the various zoonoses. The importance and recognition of wildlife as a reservoir of zoonoses are increasing. Costeffective prevention and control of these zoonoses necessitate an interdisciplinary and holistic approach and international cooperation. Surveillance, laboratory capability, research, training and education, and communication are key elements. 

Body: diseased, or dead hare or rodent. By contrast, rabies virus is transmitted by bite (saliva) from a rabid animal. Hantaviruses are spread from rodents to humans by aerosols in dust from rodent excreta. Zoonotic agents can also be spread from wildlife to humans indirectly by contaminated food and water, for example Salmonella spp. and Leptospira spp. Many zoonoses with a wildlife origin are spread through insect vectors. For example, mosquitoes are wellknown vectors of several wildlife zoonoses, such as Rift Valley fever, equine encephalitis, and Japanese encephalitis. Y. pestis can be spread by fleas, Bacillus anthracis spores by flies, and Leishmania by sand-flies, whereas ticks are essential in the spread of Borrelia burgdorferi and Ehrlichia/Anaplasma. A good example of a zoonotic agent with many different transmission modes is F. tularensis. Rodents and hares constitute the main sources of infections, and hunters are at particular risk of acquiring the disease. The transmission mode also affects the clinical manifestation in humans. The agent can be transmitted by direct contact through the handling of an infected carcass and through tick or mosquito bites, which cause initial skin symptoms such as ulcers. Infection may also occur after eating insufficiently cooked meat from an infected animal or contaminated drinking water, causing symptoms from the digestive tract, and by inhalation of contaminated dust, causing a pneumonialike illness. Salmonella spp. can also be spread from wildlife to humans in different ways. Reptile-associated salmonellosis is a well-described phenomenon, especially among children. The increasing popularity of keeping reptiles and other exotic animals as pets presents a public health problem, as such animals are commonly carriers of Salmonella and thereby can infect humans directly or indirectly. In Norway, special types of Salmonella enterica subsp. enterica serovar Typhimurium (S. Typhimurium) occur endemically in hedgehogs and wild passerine birds, causing sporadic cases and small outbreaks in humans. In 1987, a nationwide outbreak of S. Typhimurium infections was traced to chocolate bars that had been contaminated by wild birds in the factory. In 1999, a waterborne outbreak of S. Typhimurium infections was linked to a dead seagull that had contaminated a reservoir water source from which the water was used untreated (8) (9) (10) . B. anthracis, the etiologic agent of anthrax, primarily a disease of herbivores, can also be transmitted from wildlife to humans by various modes. The spores formed by the bacteria are very resistant and have been found to remain dormant and viable in nature for >100 years (11) . Anthrax is spread by food and water contamination or by the spread of spores by flies, vultures, and other scavengers. Humans can be infected by eating meat from infected carcasses or drinking contaminated water, through the skin by contact with infected material or by insect bites, and through the lungs by inhaling spores. Although livestock anthrax is declining in many parts of the world, the disease remains enzootic in many national parks, for example, in southern Africa and North America. Anthrax in wildlife represents a persistent risk for surrounding livestock and public health (12) . The ecologic changes influencing the epidemiology of zoonoses with a wildlife reservoir can be of natural or anthropogenic origin. These include, but are not limited to, human population expansion and encroachment, reforestation and other habitat changes, pollution, and climatic changes. The spirochete Borrelia burgdorferi, which causes Lyme borreliosis, has its main reservoir among small rodents and deer and uses various Ixodes species as vectors (13) . Lyme borreliosis was first recognized in Lyme, Connecticut, in 1975, and since then, an increasing number of cases have been reported in North America, Europe, and Asia. The increasing incidence of Lyme borreliosis in the northeastern United States in recent years can be explained by reforestation that has favored transmission of the disease through increased populations of white-tailed deer and deer mice and abundance of the tick vector, Ixodes scapularis. Wild rodents also constitute a reservoir of hantaviruses (14) . The viruses are shed in urine, droppings, and saliva, and humans are mainly infected aerogenically by inhaling aerosols containing the virus. Precipitation, habitat structure, and food availability are critical environmental factors that affect rodent population dynamics as well as viral transmission between animals and subsequently the incidence of human infection. The deer mouse is a reservoir host for Sin Nombre hantavirus, which causes hantavirus pulmonary syndrome in the southwestern United States. Because of climatic changes with increased rainfall in recent years, host abundance, and thereby spread of the pathogen, has increased, with subsequent transmission to humans. The movement of pathogens, vectors, and animal hosts is another factor influencing the epidemiology of zoonoses with a wildlife reservoir. Such movement can, for example, occur through human travel and trade, by natural movement of wild animals including migratory birds, and by anthropogenic movement of animals. For instance, infectious agents harbored within insects, animals, or humans can travel halfway around the globe in <24 hours in airplanes. Thus, infectious agents can be transported to the farthest land in less time than it takes most diseases to incubate. The appearance of West Nile virus infection in New York in 1999, and the subsequent spread within the United States, is an example of introduction and establishment of a pathogen that apparently originated in the Middle East (15) . Movement of infected wild and domestic animals is an important factor in the appearance of rabies in new locations. Rabies virus, which is widely distributed and affects various animals, especially canids, was introduced into North America by infected dogs in the early 18th century, with subsequent spillover to a variety of wild terrestrial mammals. Rabies became established in raccoons in the mid-Atlantic states in the late 1970s when raccoons were translocated from the southeastern United States, where rabies was endemic in this species (16) . Finland experienced an outbreak of rabies linked to raccoon dogs in 1988. The raccoon dog had spread to Finland after this species was released in western Russia for fur trade. Rabies most probably arrived in Finland by wolves migrating from Russia during wintertime along the ice-packed coast (17) . In the Arctic, the ice links the continents together. The movement of the arctic fox from the archipelago of Spitzbergen to Novaja Zemlja in Siberia and from Canada to Greenland has been described, indicating another way that rabies can be spread to new areas (18, 19) . Bovine tuberculosis caused by Mycobacterium bovis is another zoonosis in which both natural and anthropogenic movement of animals has influenced the epidemiology. This zoonosis is emerging in wildlife in many parts of the world, and wildlife can represent a source of infection for domestic animals and humans. Bovine tuberculosis was probably introduced into Africa with imported cattle during the colonial era and thereafter spread to and became endemic in wildlife (20) . In Ireland and Great Britain, badgers maintain the infection, whereas the brushtail possum constitutes a main wildlife reservoir in New Zealand. In parts of Michigan, bovine tuberculosis is endemic among white-tailed deer, whereas in Europe, both wild boars and various deer species can be a reservoir of the pathogen. The natural movement of these reservoir animals increases the spread of the disease to domestic animals and thereby its public health impact (21) . The epidemiology of multilocular echinococcosis, caused by the small tapeworm Echinococcus multilocularis, has also been influenced by the translocation of animals. During the summer of 2003, an outbreak of monkeypox occurred in the United States with 37 confirmed human cases (24) . Monkeypox is a rare zoonosis caused by a poxvirus that typically occurs in Africa. It was first found in monkeys in 1958 and later on in other animals, especially rodents. The African squirrel is probably the natural host. Transmission to humans occurs by contact with infected animals or body fluids. The cases in the United States, the first outside Africa, were associated with contact with infected prairie dogs. The outbreak was epidemiologically linked to imported African rodents from Ghana. Most likely, infected imported rodents have transmitted the virus to prairie dogs in United States. This transmission illustrates how non-native animal species can create serious public health problems when they introduce a disease to native animal and human populations. Thus, the transportation, sale, or distribution of animals, or the release of animals into the environment, can represent a risk for spread of zoonoses. Microbial changes or adaptation also influence the epidemiology of zoonoses with a wildlife reservoir. These changes include mutations, such as genetic drift in viruses; activation and silencing of genes; genetic recombinations, such as genetic shift in viruses; and conjugation, transformation, and transduction in bacteria. Natural selection and evolution also play a role. Transmission of adaptive or genetically changed microorganisms from wildlife to humans, either directly or indirectly through domestic animals, may occur in many ways. In this respect an interna- tional wildlife trade, often illegal, in which wild animals end up in live-animal markets, restaurants, and farms, is important because such practices increase the proximity between wildlife, domestic animals, and humans (25) . Severe acute respiratory syndrome (SARS) is a current example of likely microbial adaptation. This viral respiratory illness, caused by SARS-associated coronavirus, is be


3. PERSPECTIVE 1002 Emerging Infectious Diseases • www 

Abstract: The global trade in wildlife provides disease transmission mechanisms that not only cause human disease outbreaks but also threaten livestock, international trade, rural livelihoods, native wildlife populations, and the health of ecosystems. Outbreaks resulting from wildlife trade have caused hundreds of billions of dollars of economic damage globally. Rather than attempting to eradicate pathogens or the wild species that may harbor them, a practical approach would include decreasing the contact rate among species, including humans, at the interface created by the wildlife trade. Since wildlife marketing functions as a system of scale-free networks with major hubs, these points provide control opportunities to maximize the effects of regulatory efforts. 

Body: T hreats to global health and risk factors for emerging infectious diseases run the gamut from climate change to poverty to security issues, but few are as immediately manageable as the global trade in wildlife. Trade in wildlife provides disease transmission mechanisms at levels that not only cause human disease outbreaks but also threaten livestock, international trade, rural livelihoods, native wildlife populations, and the health of ecosystems. Quantifying the global wildlife trade is almost impossible since it ranges in scale from local barter to major international routes, and much is conducted illegally or through informal networks. Some estimates indicate that ≈40,000 live primates, 4 million live birds, 640,000 live reptiles, and 350 million live tropical fish are traded globally each year (1) . Live wildlife in markets in Guangzhou, China, trade in masked palm civets, ferret badgers, barking deer, wild boars, hedgehogs, foxes, squirrels, bamboo rats, gerbils, various species of snakes, and endangered leopard cats, along with domestic dogs, cats, and rabbits (2) . After the outbreak of severe acute respiratory syndrome (SARS) in 2003, 838,500 wild animals were reportedly confiscated from the markets in Guangzhou (3). Wild mammals, birds, and reptiles flow daily through trading centers, where they are in contact with persons and with dozens of other species before they are shipped to other markets, sold locally, or even freed and sent back into the wild as part of religious customs such as merit release (4) or because they become unwanted pets. In a single market in North Sulawesi, Indonesia, up to 90,000 mammals are sold per year (5) . In a survey conducted at 1 market in Thailand for 25 weekends, >70,000 birds, representing of 276 species, were sold (6). A similar survey of 4 markets in Bangkok in 2001 found that of 36,537 observed birds; only 37% were native to Thailand, while 63% were nonnative species (7) . In lieu of precise trade data, we conservatively estimated that in East and Southeast Asia, tens of millions of wild animals are shipped each year regionally and from around the world for food or use in traditional medicine. The estimate for trade and local and regional consumption of wild animal meat in Central Africa alone is >1 billion kg per year (8) , and estimates for consumption in the Amazon Basin range from 67 to 164 million kilograms annually (9,10); for mammals alone, this consumption consists of 6.4 million to 15.8 million individual animals (11) . In Central Africa, estimates of the number of animals consumed by humans annually vary, but 579 million has been proposed (12) . Hunters, middle marketers, and consumers experience some type of contact as each animal is traded. Other wildlife in the trade is temporarily exposed, and domestic animals and wild scavengers in villages and market areas consume the remnants and wastes from the traded and potentially traded wildlife. These numbers combined suggest that at least some multiple of 1 billion direct and indirect contacts among wildlife, humans, and domestic animals result from the wildlife trade annually. The increasingly global scope of this trade, coupled with rapid modern transportation and the fact that markets serve as network hubs rather than as product endpoints, dramatically increases the movement and potential cross-species William B. Karesh,* Robert A. Cook,* Elizabeth L. Bennett,* and James Newcomb † transmission of the infectious agents that every animal naturally hosts. Since 1980, >35 new infectious diseases have emerged in humans (13), ≈1 every 8 months. The origin of HIV is likely linked to human consumption of nonhuman primates (14) . Recent Ebola hemorrhagic fever outbreaks in humans have been traced to index patient contact with infected great apes that are hunted for food (15) . SARSassociated coronavirus has been associated with the international trade in small carnivores (16) , and a study comparing antibody evidence of exposure to this coronavirus demonstrated a dramatic rise from low or zero prevalence of civets at farms to an approximately 80% prevalence in civets tested in markets (17) . The inadvertent movement of infectious agents due to the wildlife trade is not limited to human pathogens but also affects pathogens of domestic animals and native wildlife. H5N1 type A influenza virus was recently isolated from 2 mountain hawk eagles illegally imported to Belgium from Thailand (18) . A paramyxovirus highly pathogenic for domestic poultry entered Italy through a shipment of parrots, lovebirds, and finches imported from Pakistan for the pet trade (19) . Monkeypox was introduced to a native rodent species and subsequently to humans in the United States by importing wild African rodents from Ghana for the US pet trade (20) . Chytridiomycosis, a fungal disease now identified as a major cause of the extinc-tion of 30% of amphibian species worldwide, has been spread by the international trade in African clawed frogs (21) . Merit release of wild birds and reptiles that have passed through markets provides another avenue for introducing novel infectious agents into the wild (4) and warrants further attention (Figure) . Many diseases are transmitted through the same species of parasites carried by imported animals. For example, from November 1994 to January 1995, US Department of Agriculture personnel inspected 349 reptile shipments from 22 countries containing 117,690 animals. Ticks were removed from animals in 97 shipments, and infested shipments included 54,376 animals (22). Ticks carry many diseases that threaten livestock and human health, including heartwater disease, Lyme disease, and babesiosis. The possibility of emerging infectious diseases spreading between persons and animals is rising, fueled by human activities ranging from the handling of bushmeat and the trade in exotic animals to the destruction or disturbance of wild habitat (23) (24) (25) . In a list of 1,415 human pathogens, 61% are known to be zoonotic, and multiple host pathogens are twice as likely to be associated with an emerging infectious disease of humans (26) . Seventy seven percent of pathogens found in livestock are shared with other host species (27) . In addition to the direct health effects of the pathogens on persons and animals, animal-related disease outbreaks have caused hundreds of billions of dollars of economic damage globally, destabilizing trade and producing devastating effects on human livelihoods. The rash of emerging or reemerging livestock disease outbreaks around the world since the mid 1990s, including bovine spongiform encephalopathy, foot-and-mouth disease, avian influenza, swine fever, and other diseases, has cost the world's economies $80 billion (28) . In early 2003, the United Nation's Food and Agriculture Organization reported that more than one third of the global meat trade was embargoed as a result of mad cow disease, avian influenza, and other livestock disease outbreaks. Efforts to control the spread of avian influenza in Asian countries since 2003 have required the culling of >140 million chickens (29) . The projected growth of industrial livestock production in nonindustrialized countries to meet global protein demand will increase the impact of future disease outbreaks on economic and food supply security. Some of these outbreaks will inevitably be linked to the trade in wildlife. Rather than attempting to eradicate pathogens or the wild species that may harbor them, a practical approach to decrease the risk for the spread of infectious diseases would include decreasing contact among species. Closing down retail poultry markets in Hong Kong for 1 day per month reduced the rate of H9N2 avian influenza virus in market birds (30) . Little equivalent research has been conducted in market systems that sell wildlife, but an analogous approach to the precautionary principle (31) would be an appropriate action to take before the next outbreak or pandemic. Since wildlife markets are a system of networks with major hubs, these trading points provide practical control opportunities to maximize the effects of regulatory efforts (32) . Focusing efforts at markets to regulate, reduce, or in some cases, eliminate the trade in wildlife could provide a cost-effective approach to decrease the risks for disease for humans, domestic animals, wildlife, and ecosystems. Dr. Karesh is the director of the Wildlife Conservation Society's Field Veterinary Program and also serves as the cochair of the World Conservation Union Veterinary Specialist Group. His projects and programs focus on the links between wildlife, domestic animals, and human health and well-being in nonindustrialized countries around the world.

In [ ]:
tfDTM['sum']=tfDTM['social']+tfDTM['econom']+ tfDTM['risk']+tfDTM['coronavirus']\
                +tfDTM['reduct']+tfDTM['sustain']+tfDTM['strategi']

so=tfDTM.sort_values(by='sum',ascending=False).index
d=[]
c=[]
t=[]
for k in so[:5]:
    d.append(daaw['body'].iloc[k])
    c.append(daaw['abstracts'].iloc[k])
    t.append(daaw['title'].iloc[k])
print(len(' '.join(c)))


In [ ]:
for i in range(0,3):
    print(str(i+1)+'.',t[i],'\n')
    print('Abstract:',c[i],'\n')
    print('Body:',d[i][:10000])
    print('\n')

## Socio-Economic Causes,Risks Factors and reduction Strategies.

1. Social Support and Response to AIDS and Severe Acute Respiratory Syndrome 

Abstract: Negative public reactions to emerging infectious diseases can adversely affect population health. We assessed whether social support was associated with knowledge of, worry about, and attitudes towards AIDS and severe acute respiratory syndrome. Our fi ndings suggest that social support may be central to our understanding of public responses to emerging infectious diseases. 

Body: The ability of public health institutions to effectively manage emerging infectious diseases (EIDs) and mitigate their consequences is partly a function of public reaction to an epidemic. Negative reactions may vary from denial to panic to stigmatization. Denial or minimization of the threat of an EID by the population at risk can impede prevention efforts and increase transmission. Alternatively, an overreaction to the threat of an EID can overwhelm public health systems and resources, divert resources from effective disease control activities, and lead to severe economic losses in areas affected by the disease. Stigmatization can increase problems of persons with the disease and facilitate transmission because persons with or at risk for the disease may avoid seeking healthcare and because governments may attempt to suppress information about EIDs, considering their potentially severe economic consequences. From a public health policy perspective, identifying personal characteristics and resources that predict responses of persons to EIDs is important to improve the ability of the public to learn about, rationally appraise the threat of, and minimize stigmatization of EIDs. Determinants of a person's knowledge, worry, and stigma about EIDs are poorly understood. Although the quality of information conveyed to the public by various sources (e.g., healthcare providers, media) and epidemiologic characteristics of the disease (e.g., mode of transmis-sion, case fatality rate) are important, early empirical work suggests that responses to EIDs are infl uenced by characteristics that include a greater stock of personal resources. We recently reported that less education was associated with being poorly informed about severe acute respiratory syndrome (SARS), very worried about AIDS, and having more stigmatizing attitudes toward AIDS and SARS (1, 2) . Social relationships may also be important. Helleringer et al. reported that social interactions among friends, peers, relatives, and community members may infl uence perceptions of HIV/AIDS risk and serve "as a resource for individuals to learn about and evaluate new behavioral strategies in the face of the epidemic" (3) . In this study, we assessed whether social support was independently associated with knowledge of, worry about, and attitudes toward AIDS and SARS in a representative sample of persons living in the New York City metropolitan area. Social support is defi ned as the functional aspect of social relationships (4). Data for this study came from a cohort of persons >18 years of age who were living in the New York City metropolitan area on September 11, 2001 . The cohort was recruited through a random digit dial telephone survey from March 25 through June 25, 2002. Contact information was obtained and follow-up interviews were conducted from September 25, 2002, through January 31, 2003, and from September 24, 2003, through February 29, 2004. Data regarding knowledge of, worry about, and attitudes toward AIDS and SARS were collected among a randomly selected subset of participants in the second cohort follow-up. The total sample included in this analysis, for whom data about AIDS, SARS, and social support were incomplete, consisted of 914 persons; response rate for eligible participants was 56%. Each participant received a $10 incentive. Additional details on sampling are provided elsewhere (5) . The Institutional Review Board of the New York Academy of Medicine reviewed and approved this study. We collected information about respondent sex, race/ ethnicity, age, educational attainment, marital status, and household income at baseline. Social support was assessed by using a 5-item modifi ed version of the Medical Outcomes Study social support scale (6) ; this abbreviated scale has a Cronbach α of 0.90 (7) . Social support was categorized as low, medium, or high, on the basis of tertiles of support reported in the sample. We assessed respondents' knowledge of, worry about, and attitudes toward AIDS and SARS (Table 1) . Further details on measures are provided elsewhere (1,2,5). We described sociodemographic characteristics and level of social support of the respondents and created multivariable models predicting knowledge of, worry about, and stigmatizing attitudes toward AIDS and SARS, including covariates thought to potentially confound the relationship between social support and each outcome. Analyses were weighted to correct potential selection bias related to the number of household telephones, persons in the household, and oversampling, as well as to make the sample demographically similar to the New York City metropolitan area population according to US Census 2000 estimates. We used SUDAAN software to estimate standard errors and adjust analyses for the weighting (8) . Of the 914 participants, 45.4% were male, 54.1% white, 7.6% Asian or other race, 18.5% black, and 19.8% Hispanic ( Table 2 ). In multivariable models controlling for sociodemographic variables (online Appendix Table, available from www.cdc.gov/EID/content/14/5/825-appT.htm), persons with low or medium levels of social support were signifi cantly more likely than those with high levels of social support to report being poorly informed about AIDS (p = 0.004 for low support, p = 0.038 for medium support), being very worried about AIDS (p = 0.010 for medium support) and SARS (p = 0.010 for medium support), and to express more stigmatizing attitudes toward SARS (p = 0.020 for low support). Reporting higher levels of social support was independently associated with greater knowledge of AIDS but not of SARS. Lower levels of social support were associated with more stigmatizing attitudes toward SARS but not AIDS. These patterns may be explained by differences in the epidemiology of AIDS and SARS in New York City. AIDS has been associated with tremendous illness and mortality rates in New York City (9, 10) . In contrast, the SARS epidemic was declared globally contained by the time of data collection (11) . The psychological pathways through which social support infl uences responses to EIDs may be dependent on the effect of the epidemic on the local population. For example, greater social support may be associated with greater knowledge of AIDS because, relative to SARS, AIDS is more prevalent and persons with greater social support, through their social relationships, may have greater access to information that infl uences development of knowledge about EIDs. Social support was inversely associated with worry about both AIDS and SARS, which suggested that association between social support and AIDS and SARS my persist despite differences in the epidemiology of EIDs. These fi ndings are consistent with studies showing that social interactions are associated with perceptions of the risk for HIV/AIDS (3, 12) . Several biological and psychological mechanisms have been theorized to mediate the relationship between social support and health (4). Self-effi cacy, defi ned as the degree of confi dence persons have in their ability to perform specifi c tasks, is hypothesized to be one of the primary pathways through which social support operates (4) . Therefore, social support may act as a buffer against worry about EIDs. This study has several limitations. First, covariates were treated as time-fi xed, which may have resulted in residual confounding. Second, social support is a multidimensional construct; specifi c dimensions may differentially infl uence knowledge of, worry about, and stigmatizing attitudes toward EIDs. Finally, for a cross-sectional analysis, it is diffi cult to establish temporality between levels of social support and knowledge of, worry about, and attitudes toward AIDS and SARS. Observed associations between social support and responses to EIDs may be explained by a common unmeasured covariate. Emerging Infectious Diseases • www.cdc.gov/eid • Vol. 14, No. 5, May 2008 Emerging and reemerging infectious diseases will continue to profoundly affect human health. The ability to effectively mitigate disease consequences will depend, in part, on minimizing negative public responses. We showed that social support may be central to public responses to EIDs. Further research investigating the pathways linking social support to responses to EIDs may inform interventions that help guide public health and offi cial responses to these diseases. Search past issues of EID at www.cdc.gov/eid


2. Systematic Review of Economic Evaluations of Preparedness Strategies and Interventions against Influenza Pandemics 

Abstract: Background: Although public health guidelines have implications for resource allocation, these issues were not explicitly considered in previous WHO pandemic preparedness and response guidance. In order to ensure a thorough and informed revision of this guidance following the H1N1 2009 pandemic, a systematic review of published and unpublished economic evaluations of preparedness strategies and interventions against influenza pandemics was conducted. The search was performed in September 2011 using 10 electronic databases, 2 internet search engines, reference list screening, cited reference searching, and direct communication with relevant authors. Full and partial economic evaluations considering both costs and outcomes were included. Conversely, reviews, editorials, and studies on economic impact or complications were excluded. Studies were selected by 2 independent reviewers. Results: 44 studies were included. Although most complied with the cost effectiveness guidelines, the quality of evidence was limited. However, the data sources used were of higher quality in economic evaluations conducted after the 2009 H1N1 pandemic. Vaccination and drug regimens were varied. Pharmaceutical plus non-pharmaceutical interventions are relatively cost effective in comparison to vaccines and/or antivirals alone. Pharmaceutical interventions vary from cost saving to high cost effectiveness ratios. According to ceiling thresholds (Gross National Income per capita), the reduction of nonessential contacts and the use of pharmaceutical prophylaxis plus the closure of schools are amongst the cost effective strategies for all countries. However, quarantine for household contacts is not cost effective even for low and middle income countries. 

Body: When a new subtype of influenza A virus which is infectious to humans gains human-to-human transmissibility efficiently enough to cause community level outbreaks, this virus is said to have pandemic potential. If this new subtype spreads globally causing disease and deaths, it becomes pandemic. Since the 16th century, influenza pandemics have occurred at intervals ranging between 10-50 years, creating varying levels of impact on societies. [1] In March 2009, a new subtype of influenza A H1N1 virus was identified in Mexico and the United States. It spread to all continents in less than nine weeks becoming the first pandemic of the 21st century. Children, young adults, pregnant women, and those with chronic illnesses were disproportionately affected and constituted the majority of the hospitalization cases. The estimated case fatality rate was 0.15-0.25%, with most deaths in middleaged adults with underlying diseases. [2] Although concrete evidence on the macroeconomic consequences of the 2009 H1N1 pandemic have not yet been revealed, previous studies estimated the potential reduction of labor productivity and consumption demand, with results showing an approximately 2-5% decrease in annual gross product. [3, 4] The World Health Organization (WHO) published pandemic preparedness and response guidance in 1999 with two revisions: the first in 2005 and the second in 2009. [1, 2] These documents summarize the recommended WHO and national actions against pandemic influenza according to recognized pandemic phases. For some recommendations, evidence is limited to observations or epidemiological models. In some cases, inferences are drawn from other respiratory infectious diseases, such as seasonal influenza or severe acute respiratory syndrome. With a view to incorporating important experience and evidence acquired during the H1N1 2009 pandemic, the WHO will revise its pandemic preparedness guidelines. Including cost effectiveness evidence in the revision process will strengthen the guidance by providing a framework to prioritize the allocation of limited resources in impending, high risk times. The aim of this paper is to systematically review published and unpublished economic evaluations of interventions to control and prevent human influenza pandemics. Funded by the WHO, this study describes and assesses the identified studies and determines patterns in cost utility ratios. The findings are expected to contribute to the revision of the WHO guidance on pandemic influenza, potentially support policymakers to make informed decisions on allocating resources effectively, and identify gaps for future research. which is more focused on non-pharmaceutical issues and also covers conference proceedings, was also performed. To retrieve additional grey literature reports and conference proceedings, the search was expanded by using the generic search engine Google (www.google.co.uk, on 06.10.2010/22.09.2011), and the science specific search engine Scirus (www.scirus.com, 2009-22.09.2011). Furthermore, reference lists of relevant publications were screened. and cited reference searching of the first economic evaluation on this topic [5] was also performed using Web of Science (via ISI Web of Knowledge, 1970-22.09.2011). Additional reports were obtained through correspondence between one reviewer (RPV) and authors of eligible studies and conference abstracts. The search strategies used controlled vocabulary thesaurus terms, whenever available, and relevant free text terms, including 'pandemic', 'H1N1', 'influenza', 'cost benefit', 'cost(s)', 'cost effective' and 'economic(s)', in different combinations. Free text terms in other languages were also employed when appropriate. Search strategies applied to electronic data sources, and outputs are shown in Table S1 and S2. All identified abstracts were reviewed by two independent reviewers from a review team (AM, KW, NP, RPV and SK). Discrepancies were resolved by a third reviewer (YT). The papers were included in the analysis if they met the criteria shown in Table 1 . A standardized data extraction form was devised. The articles were grouped according to type of evaluation as follows: i) a cost minimization analysis if they compared costs of different interventions with evidence of equal effectiveness; ii) a cost benefit analysis (CBA) if they measured health outcomes in monetary units; iii) a cost effectiveness analysis (CEA) if they expressed health outcomes in natural units, e.g., cases averted, hospitalizations averted, or deaths averted; and iv) a cost utility analysis (CUA) if they presented health outcomes in common units, e.g., quality adjusted life years (QALYs) or disability adjusted life years (DALYs). The studies were appraised in two different ways for quality assessment purposes following approaches employed by Teerawattananon et al. [6] First, they were assessed according to specific methodological and reporting practices for economic evaluation studies such as expression of perspective used, relationship between time horizon and discounting, reporting of incremental cost effectiveness ratios (ICERs), performing uncertainty analysis, and declaration of funding support. Second, studies were assessed according to the quality of evidence used, since it is widely recognized that the credibility of economic evaluations depends not only on the appropriateness of the methods employed but also on the input evidence. Several types of evidence are evaluated according to their level of quality: i) clinical effect sizes; ii) adverse events and complications; iii) baseline clinical data; iv) resource use; v) costs; and vi) utilities (only applicable to cost utility analyses). The data sources of each component are ranked from one to six in descending order. Rank 1 is given if parameters are derived from the most appropriate data sources. [6, 7] (For detailed information about ranking consult Table S3 ; for technical terms used in the data extraction, the definitions given by the HTA Glossary were used: http://htaglossary.net). This review compared the value for money of different interventions for the prevention and control of influenza pandemics in comparison with 'no intervention'; however, the evaluations were conducted in different settings and timeframes. This study converted cost effectiveness ratios into a common currency and utility unit. Costs in international dollars (I$), at 2011 values, were calculated using national gross domestic product deflator values and implied purchasing power parity conversion rates from the International Monetary Fund (http://www.imf. org/external/ns/cs.aspx?id = 28). In addition, exchange rates obtained from the OANDA website (http://www.oanda.com/ currency/historical-rates) were applied when cost outcomes were reported in foreign currencies rather than in local equivalents. Because of the lack of explicit and implicit thresholds for most countries to determine which interventions are cost effective, the World Bank thresholds for classifying countries into low income, lower middle income, upper middle income, and high income countries according to 2010 Gross National Income (GNI) per capita were used as maximum ceiling thresholds. [8] In other words, the interventions that spent fewer resources than the upper bound of GNI per capita given by the World Bank in order to gain one QALY or save one DALY represent good value for money. The search in the electronic databases identified a total of 677 records. In addition, 510 records were identified through internet search engines. There were 98 records that met the inclusion criteria and were assessed for eligibility, while 56 full texts were not included in the final analysis. These studies were excluded because they were reviews (14), were not economic evaluations (9), focused on seasonal influenza (6), focused on the impact of influenza (7), had no full text available for some conference abstracts (9), were not in the eligible languages (1), or did not report both the costs and outcomes of interventions (10) . In addition, eight full text papers were identified from correspondence with authors of eligible papers, cited reference searching, and reference list screening, of which six were excluded as they studied the impact of disease (2) or were not economic evaluations (4) . Finally, 44 studies were considered in our analysis ( Figure 1 ). The majority of studies adopted CEA and CUA approaches: 14 CEA, 16 CUA, 2 CEA and CBA combined, and 1 CEA and CUA combined. Six studies reported CBA results and five partial economic evaluations. Figure 2 illustrates the relationships among study setting, year of study, and year of publication. More than half of the studies (n = 23) were conducted in 2009 and 2010 after the H1N1 pandemic event. 14 studies assessed the value for money of interventions in the United States, followed by Canada (n = 5), the United Kingdom and Singapore (n = 3, each), Australia, the Netherlands, and France (n = 2, each), and another five countries with one study each. There were three studies conducted for Table 1 . Inclusion and exclusion criteria employed in the abstract selection process. NOriginal economic evaluations considering prevention or control of the 2009 human influenza pandemic or other potential human influenza pandemics NPartial economic evaluations if both costs and outcomes of one intervention, either pharmaceutical or non-pharmaceutical, were considered NFull economic evaluations if costs and outcomes of more than one pharmaceutical or non-pharmaceutical interventions were considered multinational settings, for example, one was conducted for three European countries [9] while another covered up to ten countries in diverse income groups. [10] The characteristics of the studies included in the analysis are summarized in Table S4 . Modified from the WHO and the World Bank's classification for the containment and mitigation of influenza pandemics, [52, 53] Table 2 d


3. Social history and exposure to pathogen signals modulate social status effects on gene regulation in rhesus macaques 

Abstract: Social experiences are an important predictor of disease susceptibility and survival in humans and other social mammals. Chronic social stress is thought to generate a proinflammatory state characterized by elevated antibacterial defenses and reduced investment in antiviral defense. Here, we manipulated long-term social status in female rhesus macaques to show that social subordination alters the gene expression response to ex vivo bacterial and viral challenge. As predicted by current models, bacterial lipopolysaccharide polarizes the immune response such that low status corresponds to higher expression of genes in NF-κBdependent pro-inflammatory pathways and lower expression of genes involved in the antiviral response and type I interferon (IFN) signaling. Counter to predictions, however, low status drives more exaggerated expression of both NF-κB and IFN-associated genes after cells are exposed to the viral mimic Gardiquimod. Status-driven gene expression patterns are not only linked to social status at the time of sampling, but also to social history (i.e., past social status), especially in unstimulated cells. However, for a subset of genes, we observed interaction effects in which females who fell in rank were more strongly affected by current social status than those who climbed the social hierarchy. Together, our results indicate that the effects of social status on immune cell gene expression depend on pathogen exposure, pathogen type, and social history -in support of social experience-mediated biological embedding in adulthood, even in the conventionally memory-less innate immune system. 

Body: The social environment, both in early life and in adulthood, has a profound and often long-lasting impact on health and mortality in humans and other social mammals (1) (2) (3) (4) (5) . This relationship is thought to arise in part through changes in gene regulation, which mediate the genomic response to physiological signals of social stress (e.g., glucocorticoids, adrenaline, noradrenaline; (6, 7) ). Gene expression signatures of social status and social adversity have now been reported in multiple studies, encompassing clinical and population-based samples in humans, and studies of both experimental and natural populations in other social animals (6, (8) (9) (10) (11) (12) (13) (14) (15) (16) (17) (18) (19) (see also (20) (21) (22) (23) (24) (25) for evidence in social insects and other social vertebrates). Because this work has concentrated most extensively on peripheral white blood cells, it provides a direct window into how social experiences are reflected in the regulation of the immune system (26) (27) (28) . Several broad patterns have emerged from these studies. First, high social adversity, including social isolation, early life insults, and low social status in adulthood, tends to predict higher expression of genes in pro-inflammatory pathways. This observation dovetails with associations between chronic social stress and elevated levels of protein biomarkers of inflammation, including those produced by peripheral blood cells (e.g., interleukin-6; (13, 29, 30) ). Second, high social adversity tends to predict lower expression of genes that function in the innate immune defense against virus, especially genes involved in type I interferon (IFN) signaling. In most cases, this pattern has been shown based on data in unstimulated cells (6) ; however, in bacterial lipopolysaccharide (LPS)-stimulated cells, induction of type I IFN-associated gene expression responses are also attenuated in low status (compared to high status) rhesus macaques (18) . Third, these findings are explained in part by socially patterned differences in the use of immune defense-modulating transcription factors. For example, genes that are more highly expressed in low status female rhesus macaques are enriched near accessible binding sites for the transcription factor complex nuclear factor kappa-B (NF-κB), a master regulator of inflammation. In contrast, genes that are more highly expressed in high status females fall near accessible binding sites for interferon regulatory factors, which coordinate type I IFN-mediated responses (18, 31) . These observations have led some authors to propose social environment-mediated trade-offs between antibacterial defense, associated with NF-κB-driven pro-inflammatory signaling, and antiviral defense, associated with the type I IFN response (6, 32) . Such a model argues that high social adversity shifts investment in the immune system towards resistance against bacterial pathogens and wound healing (possibly in anticipation of physical insults), at the cost of increased susceptibility to viral pathogens. This in turn accounts for both social gradients in conditions linked to chronic inflammation, such as cardiovascular disease, and social gradients in viral infections. Consistent with this hypothesis, psychosocial stress predicts reactivation rates of latent herpesvirus in mice and humans (33, 34) and rates of respiratory virus infection for experimentally exposed human subjects (35, 36) . Similarly, low status cynomolgus macaques males housed in a controlled environment experience both increased susceptibility to experimentally administered adenovirus and elevated rates of coronary artery stenosis (35, 37, 38) . Nevertheless, it remains unclear whether social patterning of viral susceptibility is directly related to social patterning of gene expression in peripheral blood cells. In particular, increased susceptibility to virus has been suggested to occur because the trade-offs induced by social adversity lead to insufficient production of antiviral gene transcripts (32) . This logic is largely based on lower expression levels for key antiviral genes (e.g., MX1, OAS family genes, interferon regulatory factors) measured at baseline. However, exposure to immune stimulants can radically change the transcriptional landscape of immune cells, and social environmental effects on gene expression have been shown to specifically depend on the cellular environment. For example, in rhesus macaque females, social status has more pronounced effects on gene expression after exposure to LPS, but weaker effects on gene expression after exposure to glucocorticoids (18, 31) . Because no study to date has evaluated the effects of social adversity on gene expression after both bacterial and viral challenge, it is therefore unclear whether chronic social stress in fact attenuates the gene regulatory response to virus, consistent with a trade-offs model. In addition, although increased expression of inflammation-related genes and decreased expression of type I IFN-related genes have been associated with social adversity in both adulthood and early life (9, 13, 18, 39) but see (40), we do not yet understand how the timing of social experiences affects the response to either pathogen type. To address these gaps, we turned to an animal model for social subordination-induced chronic stress: dominance rank in female rhesus macaques. This model takes advantage of the highly hierarchical social structure of female macaques, in which low rank predicts increased harassment and reduced social control, and combines it with the ability to manipulate rank via controlled introduction into newly formed social groups (earlier introduced females are higher ranking) (41, 42) . Using this study design, we previously showed that social status effects on blood cell gene expression are pervasive, cell typespecific, and dependent on the cellular environment: most relevant to this study, they can be substantially altered by bacterial stimulation (18) . Importantly, we also demonstrated that serial manipulations of dominance rank are possible. Specifically, by rearranging group composition a second time and placing females of previously similar rank into the same social group, the same individuals can be observed when they occupy two distinct positions in the social status hierarchy. Because pre-and post-rearrangement social status are completely uncorrelated, this approach provides an ideal setting to investigate the relative contributions of past versus present social environment to immune gene regulation. To do so here, we measured genome-wide gene expression levels in 45 adult female rhesus macaques: members of nine, five-member social groups that had undergone two serial rank manipulations spaced ~1 year apart (18, 43, 44) . We generated genome-wide expression profiles in blood at baseline, and in blood samples stimulated with bacterial and viral ligands. Finally, we took advantage of historical dominance rank data from the same females to evaluate whether, and for what genes and pathways, immune gene regulation in adulthood is influenced by biological embedding-when social experience leads to systematic, stable biological changes with the potential to influence health (45, 46) . Together, our analyses provide new insight into how social subordination-induced chronic stress -both past and present -differentially affects the molecular response to distinct pathogen types. We experimentally manipulated the dominance ranks of 45 adult female rhesus macaques by sequentially introducing them into newly constructed social groups of five females each (Fig. 1A , n=9 social groups, SI Appendix, Dataset S1), as described in (18) . We maintained these groups for ~1 year (February 2013 -March 2014: Phase I). We then rearranged group composition by performing a second series of sequential introductions, in this case designed so that females from the same or adjacent ranks in Phase I were co-housed in their new groups. We followed the rearranged groups for another 10 months (April 2014 -February 2015: Phase II). As expected from this experimental paradigm (42) , earlier introduction predicted higher social status in both phases, as measured by a continuous Elo rating score (Pearson's r between order of introduction and Elo score: Phase I: −0.57, p = 4.1 x 10 -5 ; Phase II: -0.68, p =3.3 x 10 -7 ). Importantly, once dominance ranks were established, they remained highly stable throughout each study phase, and individual Elo scores were completely uncorrelated between phases (r=0.06, p=0.68, SI Appendix, Fig. S1 ). Schematic representation of group formation in Phase I, mid-study rearrangement, and reorganization into new hierarchies in Phase II. (B) Principal components analysis of gene expression data across all three conditions. The combination of PC1, PC4 and PC5 separates unstimulated negative controls (NC, blue) from samples stimulated with either LPS (green) or Gard (yellow). (C) Number of rank-associated genes (FDR<5%) that are more highly expressed in high ranking females (top bars) or low ranking females (bottom bars), within each condition. To characterize the effects of social status on the immune response to bacterial versus viral challenge, including the contribution of social history, we obtained blood samples from each study subject in Phase II of the study. We generated three gene expression profiles per animal, from: (i) a control (untreated) sample, with blood cultured in cell culture media only (negative control, NC); (ii) a sample cultured in cell culture media spiked with LPS, a Tolllike receptor 4 (TLR4) agonist that mimics infection by Gram-negative bacteria; and (iii) a sample cultured in cell culture media spiked with Gardiquimod (Gard), a Toll-like receptor 7 (TLR7) agonist that mimics infection by a single-stranded RNA virus.

# Conclusions and Model Advantages

### In this programme the main advantage is that we can find research materials by just entering Keywords in that document.